In [28]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    #print(questions[:5])
    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('answers_squad_format.json')
#val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

In [29]:
from transformers import pipeline

pipe = pipeline("question-answering", model="twmkn9/distilbert-base-uncased-squad2")

In [30]:
train_answers[:2]

[{'text': 'Being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. Remote access could also help to better prepare service technicians with the necessary information for their tasks. Knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. Moreover, due to a shorter reaction time, the customer�s machines are up and running much faster, saving them from costly downtime.\n \nFor a machine and equipment manufacturer, using remote access also means that the same number of service technicians could support more customers or offer additional services.',
  'answer_start': 0},
 {'text': 'Cyber-physical systems (CPS) are enabling technologies which bring the virtual and physical worlds together to create a truly networked world in which intelligent objects c

In [31]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
#add_end_idx(val_answers, val_contexts)

In [32]:
train_answers[:5]

[{'text': 'Being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. Remote access could also help to better prepare service technicians with the necessary information for their tasks. Knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. Moreover, due to a shorter reaction time, the customer�s machines are up and running much faster, saving them from costly downtime.\n \nFor a machine and equipment manufacturer, using remote access also means that the same number of service technicians could support more customers or offer additional services.',
  'answer_start': 0,
  'answer_end': 779},
 {'text': 'Cyber-physical systems (CPS) are enabling technologies which bring the virtual and physical worlds together to create a truly networked world in which 

In [33]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("twmkn9/distilbert-base-uncased-squad2")


train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
#val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [34]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [35]:
len(train_encodings['input_ids'])

45

In [36]:
train_encodings['input_ids'][0]

[101,
 2108,
 2583,
 2000,
 6509,
 6304,
 19512,
 1010,
 2326,
 20202,
 2071,
 2490,
 2037,
 6304,
 2012,
 2151,
 2051,
 1998,
 2013,
 5973,
 1010,
 2302,
 9352,
 2383,
 2000,
 3942,
 1996,
 8013,
 2609,
 1012,
 6556,
 3229,
 2071,
 2036,
 2393,
 2000,
 2488,
 7374,
 2326,
 20202,
 2007,
 1996,
 4072,
 2592,
 2005,
 2037,
 8518,
 1012,
 4209,
 2055,
 1996,
 3698,
 2030,
 2291,
 7561,
 1999,
 2062,
 6987,
 25828,
 13169,
 7070,
 2051,
 2138,
 3223,
 8622,
 3033,
 2030,
 2060,
 3941,
 2064,
 3641,
 1010,
 4810,
 1998,
 2716,
 2247,
 1012,
 9308,
 1010,
 2349,
 2000,
 1037,
 7820,
 4668,
 2051,
 1010,
 1996,
 6304,
 6681,
 2024,
 2039,
 1998,
 2770,
 2172,
 5514,
 1010,
 7494,
 2068,
 2013,
 17047,
 2091,
 7292,
 1012,
 2005,
 1037,
 3698,
 1998,
 3941,
 7751,
 1010,
 2478,
 6556,
 3229,
 2036,
 2965,
 2008,
 1996,
 2168,
 2193,
 1997,
 2326,
 20202,
 2071,
 2490,
 2062,
 6304,
 2030,
 3749,
 3176,
 2578,
 1012,
 1037,
 5851,
 6556,
 4434,
 2000,
 5500,
 10394,
 1998,
 3941,
 2003,
 2036,

In [37]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. remote access could also help to better prepare service technicians with the necessary information for their tasks. knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. moreover, due to a shorter reaction time, the customers machines are up and running much faster, saving them from costly downtime. for a machine and equipment manufacturer, using remote access also means that the same number of service technicians could support more customers or offer additional services. a secure remote connection to distributed machinery and equipment is also the basis for many new concepts and services within industry 4. 0 such as predictive maintenance, where a secure connection is established to 

In [38]:
train_encodings.char_to_token(0, train_answers[0]['answer_start'])

1

In [39]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        go_back=1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - go_back)
            go_back+=1
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
#add_token_positions(val_encodings, val_answers)

In [40]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [41]:
train_encodings['start_positions'][:10]

[1, 1, 1, 148, 512, 1, 512, 323, 1, 512]

In [42]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
#val_dataset = SquadDataset(val_encodings)

In [43]:
print(type(train_dataset))

<class '__main__.SquadDataset'>


In [44]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("twmkn9/distilbert-base-uncased-squad2")

In [45]:
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()
optim = AdamW(model.parameters(), lr=5e-5)

c:\Users\NIKITA_PC\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [46]:


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    loop=tqdm(train_loader)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

model.eval()

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 2: 100%|██████████| 3/3 [11:02<00:00, 220.77s/it, loss=3.55]


DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [47]:
def get_answers(question, context):
    inputs = tokenizer.encode_plus(question, context, truncation=True, padding=True, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    
    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)

    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)

    
    
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end + 1]))    
    return answer

In [121]:
context="The Foundation of Smart Manufacturing at the Indian Institute of Technology (IIT) is a research and development center dedicated to advancing the field of smart manufacturing in India. IITs are a group of autonomous technical institutions established by the Indian government to promote higher education and research in information technology.The Foundation of Smart Manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems.Its primary goal is to develop and implement cutting-edge technologies to enhance the productivity, quality, and sustainability of Indian manufacturing industries.During the months of June-July 2021, FSM organized a Summer Online Internship.900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns.The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as \u201cAugmented Reality\u201d,\u201cMachine Learning\u201d, \u201cAutomation\u201d, and \u201cIIot\u201d.The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.Masterclass Series reviews"
question="What is IIT?"
inputs = tokenizer.encode_plus(question, context, truncation=True, padding=True, return_tensors='pt')
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
print(tokenizer.decode(input_ids[0]))
print(attention_mask[0])
# with torch.no_grad():
#     output = model(input_ids, attention_mask=attention_mask)

# answer_start = torch.argmax(output.start_logits)
# print(answer_start)
# answer_end = torch.argmax(output.end_logits)
# print(answer_end)


# answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end + 1]))    
# print(answer)

[CLS] what is iit? [SEP] the foundation of smart manufacturing at the indian institute of technology ( iit ) is a research and development center dedicated to advancing the field of smart manufacturing in india. iits are a group of autonomous technical institutions established by the indian government to promote higher education and research in information technology. the foundation of smart manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems. its primary goal is to develop and implement cutting - edge technologies to enhance the productivity, quality, and sustainability of indian manufacturing industries. during the months of june - july 2021, fsm organized a summer online internship. 900 + applications were received and 150 + students were accepted for internships in various domains such as augmented reality, machine learning, automation, iiot, realtime dashboards, and robotics. more than 45

In [48]:
context="The Foundation of Smart Manufacturing at the Indian Institute of Technology (IIT) is a research and development center dedicated to advancing the field of smart manufacturing in India. IITs are a group of autonomous technical institutions established by the Indian government to promote higher education and research in information technology.The Foundation of Smart Manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems.Its primary goal is to develop and implement cutting-edge technologies to enhance the productivity, quality, and sustainability of Indian manufacturing industries.During the months of June-July 2021, FSM organized a Summer Online Internship.900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns.The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as \u201cAugmented Reality\u201d,\u201cMachine Learning\u201d, \u201cAutomation\u201d, and \u201cIIot\u201d.The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.Masterclass Series reviews"
question="What is nmghkjkj?"
print(f"Predicted Answer: {get_answers(question, context)}")
#input_ids=test_encodings['input_ids'].to(device)
#outputs=model(test_encodings['input_ids'],test_encodings['attention_mask'])

Predicted Answer: the foundation of smart manufacturing at the indian institute of technology ( iit ) is a research and development center dedicated to advancing the field of smart manufacturing in india. iits are a group of autonomous technical institutions established by the indian government to promote higher education and research in information technology


In [49]:
context="The Foundation of Smart Manufacturing at the Indian Institute of Technology (IIT) is a research and development center dedicated to advancing the field of smart manufacturing in India. IITs are a group of autonomous technical institutions established by the Indian government to promote higher education and research in information technology.The Foundation of Smart Manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems.Its primary goal is to develop and implement cutting-edge technologies to enhance the productivity, quality, and sustainability of Indian manufacturing industries.During the months of June-July 2021, FSM organized a Summer Online Internship.900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns.The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as \u201cAugmented Reality\u201d,\u201cMachine Learning\u201d, \u201cAutomation\u201d, and \u201cIIot\u201d.The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.Masterclass Series reviews"
question="What is IIT?"
print(f"Predicted Answer: {get_answers(question, context)}")
#input_ids=test_encodings['input_ids'].to(device)
#outputs=model(test_encodings['input_ids'],test_encodings['attention_mask'])

Predicted Answer: a group of autonomous technical institutions established by the indian government to promote higher education and research in information technology


Answer of above question

Predicted Answer: a group of autonomous technical institutions established by the indian government to promote higher education and research in information technology

In [50]:
context="Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. Contact us today for implementing. Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. Under Samarth Udyog Mission of the Ministry of Heavy Industry (MHI), Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. "
question="What is Foundation of Smart Manufacturing?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: foundation for smart manufacturing ( fsm ) helps, supports, and develops smart manufacturing concepts for indian industry to witness, ideate, and try out in their plants. fsm also aims for a holistic educational curriculum and skill - building program through a vibrant incubation and administrative environment. fsm skills is a holistic platform for providing immersive training experience through live lectures, online learning, live demonstrations, live labs & self paced exercises on remotely accessible actual hardware. fsm is focussed on enabling people from becoming smart to samarth ( capable ) fraternity by creating awareness, step - wise learning and becoming an expert from domains driving industry 4. 0. academic strength of iit delhi brought interdisciplinary learnings to fsm. expert faculty members from various departments contribute to it. fsm has highly trained manpower to implement the industry 4. 0 projects in guidance with experienced faculty members and ind

Answer of above que

Predicted Answer: foundation for smart manufacturing ( fsm ) helps, supports, and develops smart manufacturing concepts for indian industry to witness, ideate, and try out in their plants. fsm also aims for a holistic educational curriculum and skill - building program through a vibrant incubation and administrative environment. fsm skills is a holistic platform for providing immersive training experience through live lectures, online learning, live demonstrations, live labs & self paced exercises on remotely accessible actual hardware. fsm is focussed on enabling people from becoming smart to samarth ( capable ) fraternity by creating awareness, step - wise learning and becoming an expert from domains driving industry 4. 0. academic strength of iit delhi brought interdisciplinary learnings to fsm. expert faculty members from various departments contribute to it. fsm has highly trained manpower to implement the industry 4. 0 projects in guidance with experienced faculty members and industry partners. m. tech and ph. d students doing research in smart manufacturing are contributing to fsm technologies and increase its knowledgebase. fsm brought industry partners from mncs operating all across the globe to bring rich experience in the smart manufacturing technologies. automation industries association ( aia ) in its quest to set up a common engineering facility center ( cefc ) at iit delhi has brought together experts from various industries in the sector of automation. these experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in india. the following firms have agreed to come together to set up the cefc. they comprise technology & investment partners, simulation & amp ; integration partners and machinery partners


In [51]:
context=" The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. Contact us today for implementing. Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. Under Samarth Udyog Mission of the Ministry of Heavy Industry (MHI), Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. "
question="Explain internship?"
print(f"Predicted Answer: {get_answers(question, context)}")


Predicted Answer: the program is entirely online and will run full - time from june 1, 2023, to july 31, 2023, for a duration of two months. the focus of the internship is on the theme of ' smart factory, ' which is a rapidly growing industry. this internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. it is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. if you are interested, don ' t miss this chance to be a part of the program and enhance your career prospects.. foundation for smart manufacturing ( fsm ) helps, supports, and develops smart manufacturing concepts for indian industry to witness, ideate, and try out in their plants. fsm also aims for a holistic educational curriculum and skill - building program through a vibrant incubation and administrative environment. fsm skills is a holistic platform for providing immersive training experience 

Predicted Answer: the program is entirely online and will run full - time from june 1, 2023, to july 31, 2023, for a duration of two months.

In [52]:

context="Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. Contact us today for implementing. Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. Under Samarth Udyog Mission of the Ministry of Heavy Industry (MHI), Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. "
question="What are contact information ?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: contact us today for implementing. call us for any query011 - 26582053, 807


In [53]:
context="IITD-AIA Foundation For Smart Manufacturing is delighted to announce the successful conclusion of our week long training program titled 'Competency Building in Industry 4.0' for industry experts from Hindustan Aeronautics Limited (HAL) in the field of smartmanufacturing. HAL has entrusted us with the task of upskilling their experts from various domains. Our carefully crafted learning sessions provided valuable insights into the major domains of Industry 4.0. Including Automation, IIoT, AugmentedReality, Robotics, AI, MLIITD-AIA Foundation For Smart Manufacturing got an opportunity to be a part of G20 Digital Economy Working Group (DEWG) in Lucknow hosted by UP Government. These type of events helps all the segments of nation like Politicians, Economic dignitaries, Business leader, Entrepreneurs, Industry Associates etc. to come on a common platform and take nation towards a bright and better future and we are proud to be participating and showcasing live demonstrations of our work to the visitors.We are happy to inform that on December 22, 2022, IITD-AIA Foundation For Smart Manufacturing presented various industry 4.0 technologies at Workshop on Automation for Smart Manufacturing organized by Automation Industry Association in Pune. Visitors were given Live Demonstrations of technologies created for smart manufacturing in the cyber-physical lab at the Indian Institute of Technology, Delhi. Here are some of the day's highlights. "
question="What on December 22, 2022?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: we are happy to inform that on december 22, 2022, iitd - aia foundation for smart manufacturing presented various industry 4. 0 technologies at workshop on automation for smart manufacturing organized by automation industry association in pune


Answr of above que

Predicted Answer: we are happy to inform that on december 22, 2022, iitd - aia foundation for smart manufacturing presented various industry 4. 0 technologies at workshop on automation for smart manufacturing organized by automation industry association in pune

In [54]:
context="IITD-AIA Foundation For Smart Manufacturing is delighted to announce the successful conclusion of our week long training program titled 'Competency Building in Industry 4.0' for industry experts from Hindustan Aeronautics Limited (HAL) in the field of smartmanufacturing. HAL has entrusted us with the task of upskilling their experts from various domains. Our carefully crafted learning sessions provided valuable insights into the major domains of Industry 4.0. Including Automation, IIoT, AugmentedReality, Robotics, AI, MLIITD-AIA Foundation For Smart Manufacturing got an opportunity to be a part of G20 Digital Economy Working Group (DEWG) in Lucknow hosted by UP Government. These type of events helps all the segments of nation like Politicians, Economic dignitaries, Business leader, Entrepreneurs, Industry Associates etc. to come on a common platform and take nation towards a bright and better future and we are proud to be participating and showcasing live demonstrations of our work to the visitors.We are happy to inform that on December 22, 2022, IITD-AIA Foundation For Smart Manufacturing presented various industry 4.0 technologies at Workshop on Automation for Smart Manufacturing organized by Automation Industry Association in Pune. Visitors were given Live Demonstrations of technologies created for smart manufacturing in the cyber-physical lab at the Indian Institute of Technology, Delhi. Here are some of the day's highlights. "
question="What is full-form DEWG?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: mliitd - aia foundation for smart manufacturing got an opportunity to be a part of g20 digital economy working group ( dewg ) in lucknow hosted by up government.


Answer of above question:
Predicted Answer: mliitd - aia foundation for smart manufacturing got an opportunity to be a part of g20 digital economy working group ( dewg ) in lucknow hosted by up government.

In [55]:
context="IITD-AIA Foundation For Smart Manufacturing is delighted to announce the successful conclusion of our week long training program titled 'Competency Building in Industry 4.0' for industry experts from Hindustan Aeronautics Limited (HAL) in the field of smartmanufacturing. HAL has entrusted us with the task of upskilling their experts from various domains. Our carefully crafted learning sessions provided valuable insights into the major domains of Industry 4.0. Including Automation, IIoT, AugmentedReality, Robotics, AI, MLIITD-AIA Foundation For Smart Manufacturing got an opportunity to be a part of G20 Digital Economy Working Group (DEWG) in Lucknow hosted by UP Government. These type of events helps all the segments of nation like Politicians, Economic dignitaries, Business leader, Entrepreneurs, Industry Associates etc. to come on a common platform and take nation towards a bright and better future and we are proud to be participating and showcasing live demonstrations of our work to the visitors.We are happy to inform that on December 22, 2022, IITD-AIA Foundation For Smart Manufacturing presented various industry 4.0 technologies at Workshop on Automation for Smart Manufacturing organized by Automation Industry Association in Pune. Visitors were given Live Demonstrations of technologies created for smart manufacturing in the cyber-physical lab at the Indian Institute of Technology, Delhi. Here are some of the day's highlights. "
question="What is HAL?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: hal has entrusted us with the task of upskilling their experts from various domains


In [56]:
context="Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. At FSM we provide various Client services and training under the domain of Smart Manufacturing. The services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions."
question="What are the service provided?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: the services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions


Predicted Answer: the services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions

In [57]:
context="Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. At FSM we provide various Client services and training under the domain of Smart Manufacturing. The services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions. The Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation Integration Partners and Machinery Partners."
question="Explain partners?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: contact us today for implementing call us for any query011 - 26582053, 8076197190 or email us oninfo @ iafsm. in. the program is entirely online and will run full - time from june 1, 2023, to july 31, 2023, for a duration of two months. the focus of the internship is on the theme of ' smart factory, ' which is a rapidly growing industry. this internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. it is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. if you are interested, don ' t miss this chance to be a part of the program and enhance your career prospects.. foundation for smart manufacturing ( fsm ) helps, supports, and develops smart manufacturing concepts for indian industry to witness, ideate, and try out in their plants. fsm also aims for a holistic educational curriculum and skill - building program through a vibrant incubat

In [58]:
context="Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of 'Smart Factory,' which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects.. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware.. Fill out the online registration form Shortlisting based on the eligibility criteria Level 1: Learning modules (Approx. time: 24 hrs) will be made available on fsmskills website. To be completed by 20th May 2023. Post your signup on fsmskill, you will gain access to the Level-1 course of IAFSM. This step involves learning of Smart Manufacturing Concepts and is the foundation course that is mandatory for all interns to qualify latest by 20th May 2023. You are required to go through recorded lectures and study the content thoroughly to clear the assignments of Level-1 in maximum 2 attempts. After 2 attempts, if you are not able to qualify for Level-1, your candidature will no longer be considered for internship.    Level 2 : After successful completion of Level-1, You will be redirected to the Stream Learning module which involves learning of specific domains of your choice. You can choose from the available domains and try out a maximum of 3 domains of your choice. You are then required to undergo and clear the assignments based on your selection and qualify the Level 2 assignments after which you shall be taken forward for Project Based Learning under Summer Internship 2023. The assignments are to be attempted and cleared latest by 28th May 2023. Internship Projects: After successful completion of Level-2, Interns will be allocated projects based on domain preference, level-2 performance and interaction with the mentors. Final Selection and Project Allocation based on the Level 2 performance and preferences These are full-time internships and require 48 hours per week    "
question="what are the process for internship?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: the program is entirely online and will run full - time from june 1, 2023, to july 31, 2023, for a duration of two months. the focus of the internship is on the theme of ' smart factory, ' which is a rapidly growing industry. this internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. it is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. if you are interested, don ' t miss this chance to be a part of the program and enhance your career prospects.. foundation for smart manufacturing ( fsm ) helps, supports, and develops smart manufacturing concepts for indian industry to witness, ideate, and try out in their plants. fsm also aims for a holistic educational curriculum and skill - building program through a vibrant incubation and administrative environment. fsm skills is a holistic platform for providing immersive training experience 

In [59]:
context="The Foundation of Smart Manufacturing at the Indian Institute of Technology (IIT) is a research and development center dedicated to advancing the field of smart manufacturing in India. IITs are a group of autonomous technical institutions established by the Indian government to promote higher education and research in information technology.The Foundation of Smart Manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems.Its primary goal is to develop and implement cutting-edge technologies to enhance the productivity, quality, and sustainability of Indian manufacturing industries.During the months of June-July 2021, FSM organized a Summer Online Internship.900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns.The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as \u201cAugmented Reality\u201d,\u201cMachine Learning\u201d, \u201cAutomation\u201d, and \u201cIIot\u201d.The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.Masterclass Series reviews"
question="What is FSM Masterclass Series?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: a 4 - month program, having 4 different masterclasses for technologies such as “ augmented reality ”, “ machine learning ”, “ automation ”, and “ iiot ”


In [60]:
context="The Foundation of Smart Manufacturing at the Indian Institute of Technology (IIT) is a research and development center dedicated to advancing the field of smart manufacturing in India. IITs are a group of autonomous technical institutions established by the Indian government to promote higher education and research in information technology.The Foundation of Smart Manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems.Its primary goal is to develop and implement cutting-edge technologies to enhance the productivity, quality, and sustainability of Indian manufacturing industries.During the months of June-July 2021, FSM organized a Summer Online Internship.900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns.The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as \u201cAugmented Reality\u201d,\u201cMachine Learning\u201d, \u201cAutomation\u201d, and \u201cIIot\u201d.The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.Masterclass Series reviews.Traditionally, manufacturing has been considered as a process that convert raw materials into physical products in the factories by managing resources with best automation practices available.Today, drivers such as technology, sustainability, optimization and the need to meet customer demands have once again encouraged the transformation of the manufacturing industry to become adaptive, fully connected, and aware of its own power quality. One of the most significant trends in manufacturing is of improved information technology solutions involving the union of conventional automation with the information technology.Cyber-physical systems (CPS) are enabling technologies which bring the virtual and physical worlds together to create a truly networked world in which intelligent objects communicate and interact with each other. They are 'enabling technologies' which make multiple innovative applications and processes a reality as the boundaries between the real and virtual worlds disappear. Cyber-physical systems provide the basis the creation of an Internet of Things (IoT) which makes smart services and products possible. A cyber-physical system (CPS) is a “thing” in the Internet of Things. It is a combination of mechanical, electronic and software components that communicate via a data infrastructure such as the Internet, react flexibly to external influences and exchange data with information systems and other CPSs. In manufacturing facilities, cyber-physical systems will communicate with intelligent, networked industrial production and logistics units-also known as cyber-physical production systems (CPPS). The CPSs exchange information, trigger actions in production and reciprocally control themselves autonomously. This enables industrial processes in manufacturing, engineering, use of materials, supply chain management and life cycle management to be fundamentally restructured and optimized."

question="What is Cyber Physical System?"
print(f"Predicted Answer: {get_answers(question, context)}")




Predicted Answer: a cyber - physical system ( cps ) is a “ thing ” in the internet of things


In [61]:
context="The Foundation of Smart Manufacturing at the Indian Institute of Technology (IIT) is a research and development center dedicated to advancing the field of smart manufacturing in India. IITs are a group of autonomous technical institutions established by the Indian government to promote higher education and research in information technology.The Foundation of Smart Manufacturing focuses on integrating information technology and manufacturing processes to create intelligent and efficient manufacturing systems.Its primary goal is to develop and implement cutting-edge technologies to enhance the productivity, quality, and sustainability of Indian manufacturing industries.During the months of June-July 2021, FSM organized a Summer Online Internship.900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns.The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as \u201cAugmented Reality\u201d,\u201cMachine Learning\u201d, \u201cAutomation\u201d, and \u201cIIot\u201d.The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.Masterclass Series reviews.Traditionally, manufacturing has been considered as a process that convert raw materials into physical products in the factories by managing resources with best automation practices available.Today, drivers such as technology, sustainability, optimization and the need to meet customer demands have once again encouraged the transformation of the manufacturing industry to become adaptive, fully connected, and aware of its own power quality. One of the most significant trends in manufacturing is of improved information technology solutions involving the union of conventional automation with the information technology.Cyber-physical systems (CPS) are enabling technologies which bring the virtual and physical worlds together to create a truly networked world in which intelligent objects communicate and interact with each other. They are 'enabling technologies' which make multiple innovative applications and processes a reality as the boundaries between the real and virtual worlds disappear. Cyber-physical systems provide the basis the creation of an Internet of Things (IoT) which makes smart services and products possible. A cyber-physical system (CPS) is a “thing” in the Internet of Things. It is a combination of mechanical, electronic and software components that communicate via a data infrastructure such as the Internet, react flexibly to external influences and exchange data with information systems and other CPSs. In manufacturing facilities, cyber-physical systems will communicate with intelligent, networked industrial production and logistics units-also known as cyber-physical production systems (CPPS). The CPSs exchange information, trigger actions in production and reciprocally control themselves autonomously. This enables industrial processes in manufacturing, engineering, use of materials, supply chain management and life cycle management to be fundamentally restructured and optimized."

question="Explain Cyber Physical System?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: cyber - physical systems provide the basis the creation of an internet of things ( iot ) which makes smart services and products possible


In [62]:
context="Additionally it demonstrates the HMI Designing and Dashboard designing feature. This is the third kit of the smart trainer modules used to demonstrate a working manufacturing system. It has a servo motor and drive to rotate an indexing table. Along with the motion feature, it has four stations which can be used to demonstrate multiple operations. Each station has smart inductive sensors to sense the part  ‘s presence which consists of the distance and proximity sensors, servo drive, test product, RFID Reader, Solenoid, USB Camera mounted on them.Station 1: It has a RFID controller used to read and write the tag values of a product. It communicates with IO Link master (present in PLC trainer kit) and IO link master communicates the data with the PLC. So, RFID operation and use case is demonstrated. Station 2: This station consists of a distance sensor used to calculate the height or distance of the product. It is a smart sensor and communicates with IO Link master. So, along with the analog value it communicates it  2019s status to the PLC as well. Verification of product dimension is also demonstrated. Station 3: This station has a vision camera installed on it. This camera is used for image processing and product testing. Sorting operation is performed using the image processing feature. Station 4: This station has a solenoid which is directly connected with the digital output of the PLC. Extension and retraction of the solenoid is programmed in the PLC for this operation.This mechanism kit is used to demonstrate and train motion configuration and programming, Vision System, Smart Sensing, Machine Learning, Augmented Reality.  Programming Kit HMI Kit Mechanism Kit It is showing the working of Smart Manufacturing Kit Education & Training Device Services Research & Skill Certification  Software-> Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Multi Process Robotic Cell demonstrates a flexible manufacturing cell. This cell is an assembly line where components of a main part gets assembled, the main part comprises of minimum of 7 sub-parts and more, which are then assembled by placing, inserting and part-screwing the main part. It has multiple stations which performs operation on the parts (like valve body assembly, vision inspection, testing, packaging). The main feature of the cell is exhibited using the programming, control and diagnosis of the robotic manipulator amongst other peripheral and safety devices. We call it as Multi Process robotic cell because it can be used for more than one manufacturing process.  There are different stations in the robotic cell: 1. Conveyor Station: Use to transfer the inventory tray or pallet to the place where robot can pick them.2. Assembly Station: All the parts (axisymmetric and prismatic parts) will be assembled and screwed on the valve body in this station. Along with that inventory scanning and vision inspection of the valve bodies will be done.3.Testing Station: All the pneumatic ports of the valve body will be tested in this station. If there is any leakage,"
question="What is Staion1?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: station 1 : it has a rfid controller used to read and write the tag values of a product. it communicates with io link master ( present in plc trainer kit ) and io link master communicates the data with the plc. so, rfid operation and use case is demonstrated. station 2 : this station consists of a distance sensor used to calculate the height or distance of the product. it is a smart sensor and communicates with io link master


In [63]:
context="the best possible way. These were designed, manufactured and integrated in FSM. Modules/Trainings covered in these kits: PLC Programming, Network Protocols HMI Designing Data Logging Dashboard Designing, NodeRed IIOT gateways and platforms Augmented Reality Machine Learning Smart Sensing Motion configuration and programming Vision System  This is one of the kits of the smart manufacturing trainer module. This kit has the brain of the other two kits (HMI Kit and Mechanism Kit) as programming is done in this kit. It has a PLC for programming and digital inputs/outputs, analog inputs/outputs to connect the required sensors with the PLC. It has an IO link module to connect the smart sensors, so alongwith the 24VDC or analog input this module communicates the sensor status with the PLC. Data logging and data collection is one of the critical steps for IIOT and for that we have integrated IOT gateway and other software platforms in this kit. To connect sensors with the PLC, banana connectors are used. It can communicate on a number of network protocols and it has the feature to communicate with multiple HMIs and Servo drives. In programming kit we have PLC hardwares from Rockwell, Siemens, Schneider, Omron, Delta, Turck and Mitsubishi for programming and various softwares. NUC is also integrated in the kit so that it can be used as CPU for softwares and PLC programming. So, this kit is more than sufficient to demonstrate the features of the PLC, gateways and software platforms (Data Logging, Augmented Reality ,IIOT gateways and platforms), Smart Sensing . Human Machine Interface is used to monitor and access the PLC data. Screens can be created depending on the application.This is the second kit of the smart manufacturing trainer module. It has HMI to display the screens and monitor the PLC tags. The hardware buttons and LEDs as well which are connected with the PLC trainer kit using banana connectors, mostly like plug and play. These are used to test the PLC program and as input and output in the PLC program. HMIs used in these kits are smart HMIs which have smart features like OPC UA server/client, MQTT compatibility, user authentication on the screens, act as an edge device etc. So, kit is used to visualise the PLC logic. Additionally it demonstrates the HMI Designing and Dashboard designing feature. This is the third kit of the smart trainer modules used to demonstrate a working manufacturing system. It has a servo motor and drive to rotate an indexing table. Along with the motion feature, it has four stations which can be used to demonstrate multiple operations. Each station has smart inductive sensors to sense the part  ‘s presence which consists of the distance and proximity sensors, servo drive, test product, RFID Reader, Solenoid, USB Camera mounted on them.Station 1: It has a RFID controller used to read and write the tag values of a product. It communicates with IO Link master (present in PLC trainer kit) and IO link master communicates the data with the PLC. So, RFID operation and use case is demonstrated. Station 2: This station consists of a distance sensor used to calculate the height or distance of the product. It is a smart sensor and communicates with IO Link master. So, along with the analog value it communicates it  2019s status to the PLC as well. Verification of product dimension is also demonstrated. Station 3: This station has a vision camera installed on it. This camera is used for image processing and product testing. Sorting operation is performed using the image processing feature. Station 4: This station has a solenoid which is directly connected with the digital output of the PLC. Extension and retraction of the solenoid is programmed in the PLC for this operation.This mechanism kit is used to demonstrate and train motion configuration and programming, Vision System, Smart Sensing, Machine Learning, Augmented Reality.  Programming Kit HMI Kit Mechanism Kit It is showing the working of Smart Manufacturing Kit Education & Training Device Services Research & Skill Certification  Software-> Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Multi Process Robotic Cell demonstrates a flexible manufacturing cell. This cell is an assembly line where components of a main part gets assembled, the main part comprises of minimum of 7 sub-parts and more, which are then assembled by placing, inserting and part-screwing the main part. It has multiple stations which performs operation on the parts (like valve body assembly, vision inspection, testing, packaging). The main feature of the cell is exhibited using the programming, control and diagnosis of the robotic manipulator amongst other peripheral and safety devices. We call it as Multi Process robotic cell because it can be used for more than one manufacturing process.  There are different stations in the robotic cell: 1. Conveyor Station: Use to transfer the inventory tray or pallet to the place where robot can pick them.2. Assembly Station: All the parts (axisymmetric and prismatic parts) will be assembled and screwed on the valve body in this station. Along with that inventory scanning and vision inspection of the valve bodies will be done.3.Testing Station: All the pneumatic ports of the valve body will be tested in this station. If there is any leakage, valve body will be rejected.4. Hex-nut Assembly Station:- Hex nut will be assembled on the body in this station.5. Packaging Station: Assembled valve bodies will be placed in a box and packaging is done. A printing head will print the required details( type of bodies, number of bodies, date of assembly etc) on the box and then the box will be dispatched.  The video here demonstrates the standalone function of loading inventory using trays from conveyor station to assembly station. The tray has multiple arrays of different sized part but same variety. There are trays with 3/2, 5/2, 5/2 double solenoid, 5/3 solenoid valve body. The end-effector of the robot have more than one tool attachments. This is possible due to tool changer allowing robot to perform multiple actions like pick and place, part screwing, gripping via vacuum suction cup, bar code part scanning. Training Site Integration Simulation & Testing Prototyping <-Cyber Physical assembly line Robotic Welding Cell-> Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in To create user defined webpages to control and monitor different devices (Appliances,machines) present in CP lab. 01Remote Control and monitoring of assembly line, Compressor and MPRC 02Remote control and monitoring of domestic appliances (lights,fans,AC) 03User defined access to control the machines remotely 04Activity logging of users 05Remote Control of the training Kits to train people remotely  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Robotic Welding Cell demonstrates a flexible manufacturing cell meaning it has an inventory of its own, material handling system, variety of parts with dynamic-order.This welding cell can be a part of production line and act as standalone system.Use of single or multiple robots may involve use of a PLC also.This cell uses a saftey PLC.This cell has all these features and the basis of selection can be done in a way that the cell will be programmed, utilization of peripheral devices (proximity sensors, pneumatic cylinders), safety devices and Automatic Identification & Data Capturing (AIDC) devices like RFID, vision camera etc. The cell has following level of flexibility but the decision can be made on the basis of the flexibility required as per the following parameters:1. Variety of parts for simultaneous production2. Change of schedule3. Ready for new partWorkstations and layout design :Workstations in the cell are as per the following given applications and subject to following parameters : 1. Load/Unload stations for the cell: It is the area where the raw materials are incepted in/out of the cell.2. Inventory storage area: An inventory storage where the storage of the raw material is kept.3. Inspection area: An area where the material post operations are inspected for quality. This is kept as a separate unit away from the cell but it can be an integral part of the cell itself.4. Operations area: An area where the operations like welding is performed. In the cell , this area is more of a workspace since two robots are responsible for performing welding on the material. And it can be anywhere in the robot  ‘s workspace. 5. Layout design: It is an integral part before identifying the robots and environment preparation and also plays a key role in safety of the cell.CAD Design & Simulation:The 3D visualization and detailing of the cell is done using a designing software. The robotic path simulation software asists in defining the best possible, collision free environment that is going to be the basis of the layout design and all other hardware that are going to be installed in the cell. Apart from designing & fabrication of the cell it helps in performing the robot simulations to generate offline program, collision free environment (this is how the level of flexibility achieved) for robots at operating stage and generating Augmented Reality applications. AutomationIIoTRoboticsAugmented Reality 01Machine as a service 04Real-Time Health Monitoring of Assets in Factory 02Customer Order Proessing 03Job work and research Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in FSM is providing an exciting opportunity for 100 individuals to participate in a summer internship program. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of \Smart Factory,\ which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don  ‘t miss this chance to be a part of the program and enhance your career prospects. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0 info@iafsm.in Level 2 : After successful completion of Level-1,  "

In [64]:
question="What is Modules/Trainings covered in the kit?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: plc programming, network protocols hmi designing data logging dashboard designing, nodered iiot gateways and platforms augmented reality machine learning smart sensing motion configuration and programming vision system


In [65]:
question="Explain Station 1?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: data logging and data collection is one of the critical steps for iiot and for that we have integrated iot gateway and other software platforms in this kit. to connect sensors with the plc, banana connectors are used. it can communicate on a number of network protocols and it has the feature to communicate with multiple hmis and servo drives. in programming kit we have plc hardwares from rockwell, siemens, schneider, omron, delta, turck and mitsubishi for programming and various softwares. nuc is also integrated in the kit so that it can be used as cpu for softwares and plc programming. so, this kit is more than sufficient to demonstrate the features of the plc, gateways and software platforms ( data logging, augmented reality, iiot gateways and platforms ), smart sensing. human machine interface is used to monitor and access the plc data. screens can be created depending on the application. this is the second kit of the smart manufacturing trainer module. it has hmi 

In [66]:
question="what is Station 2?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: data logging and data collection is one of the critical steps for iiot and for that we have integrated iot gateway and other software platforms in this kit. to connect sensors with the plc, banana connectors are used. it can communicate on a number of network protocols and it has the feature to communicate with multiple hmis and servo drives. in programming kit we have plc hardwares from rockwell, siemens, schneider, omron, delta, turck and mitsubishi for programming and various softwares. nuc is also integrated in the kit so that it can be used as cpu for softwares and plc programming. so, this kit is more than sufficient to demonstrate the features of the plc, gateways and software platforms ( data logging, augmented reality, iiot gateways and platforms ), smart sensing. human machine interface is used to monitor and access the plc data. screens can be created depending on the application. this is the second kit of the smart manufacturing trainer module


In [67]:

question="why  it is called as  Multi Process robotic cell ?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: 


In [68]:
context="At FSM we provide various Client services and training under the domain of Smart Manufacturing. The services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. Contact us today for implementing. Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. Under Samarth Udyog Mission of the Ministry of Heavy Industry (MHI), Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. The collaboration is also aimed at developing a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. As part of our commitment to make FSM infrastructure available for the student community to explore and learn smart manufacturing technologies, FSM is offering 100 nos. 2-Months full-time Online Summer Internship from 01-June-2023 to 31-July-2023 under the theme of  Smart Factory . "
question="what is CEFC ?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: fsm brought industry partners from mncs operating all across the globe to bring rich experience in the smart manufacturing technologies. automation industries association ( aia ) in its quest to set up a common engineering facility center


In [69]:
context="At FSM we provide various Client services and training under the domain of Smart Manufacturing. The services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. Contact us today for implementing. Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. Under Samarth Udyog Mission of the Ministry of Heavy Industry (MHI), Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. The collaboration is also aimed at developing a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. As part of our commitment to make FSM infrastructure available for the student community to explore and learn smart manufacturing technologies, FSM is offering 100 nos. 2-Months full-time Online Summer Internship from 01-June-2023 to 31-July-2023 under the theme of  Smart Factory . "
question="what is date for internship ?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: fsm has highly trained manpower to implement the industry 4. 0 projects in guidance with experienced faculty members and industry partners


In [168]:
context="At FSM we provide various Client services and training under the domain of Smart Manufacturing. The services will enable clients to test and debug problems before installing the solution at the site & also allow them to experiment and innovate with an appropriate mix of standard and customized solutions. Foundation for Smart Manufacturing (FSM) helps, supports, and develops Smart Manufacturing concepts for Indian Industry to witness, ideate, and try out in their plants. FSM also aims for a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. FSM Skills is a holistic platform for providing immersive training experience through Live lectures, Online learning, Live demonstrations, Live Labs & Self paced exercises on remotely accessible actual hardware. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0. Academic strength of IIT Delhi brought interdisciplinary learnings to FSM. Expert Faculty members from various departments contribute to it. FSM has highly trained manpower to implement the Industry 4.0 projects in guidance with experienced faculty members and industry partners. M.Tech and Ph.D students doing research in Smart manufacturing are contributing to FSM technologies and increase its knowledgebase. FSM brought industry partners from MNCs operating all across the globe to bring rich experience in the smart manufacturing technologies. Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along with various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners. Contact us today for implementing. Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. Under Samarth Udyog Mission of the Ministry of Heavy Industry (MHI), Govt. of India, IIT Delhi, and Automation Industry Association (AIA) together with Industry sponsors have set up common engineering facilities under the aegis of the IITD - AIA Foundation for Smart Manufacturing (FSM). These facilities are meant to demonstrate, support, and develop Smart Manufacturing concepts for Indian Industry & try them out in their own plants. The collaboration is also aimed at developing a holistic educational curriculum and skill-building program through a vibrant incubation and administrative environment. As part of our commitment to make FSM infrastructure available for the student community to explore and learn smart manufacturing technologies, FSM is offering 100 nos. 2-Months full-time Online Summer Internship from 01-June-2023 to 31-July-2023 under the theme of  Smart Factory . Registration Open. Last day to register. Short Listing. Orientation. Level 1 Learning Ends. Level 2 Technology Stream Learning Ends. Final Selection and Project Allocation. Orientation 2nd Programme & Internship Starts. Internship Ends.  Registration.  Short Listing.  Level 1 Learning.  Level 2 Technology Stream Learning . Final Selection.  Project Allocation.  Internship Commencement .  Fill out the online registration form. Shortlisting based on the eligibility criteria Level 1: Learning modules  (Approx. time: 24 hrs) will be made available on fsmskills website. To be completed by 20th May 2023.Post your signup on fsmskill, you will gain access to the Level-1 course of IAFSM. This step involves learning of Smart Manufacturing Concepts and is the foundation course that is mandatory for all interns to qualify latest by 20th May 2023. You are required to go through recorded lectures and study the content thoroughly to clear the assignments of Level-1 in maximum 2 attempts. After 2 attempts, if you are not able to qualify for Level-1, your candidature will no longer be considered for internship. Level2 : After successful completion of Level-1, You will be redirected to the Stream Learning module which involves learning of specific domains of your choice. You can choose from the available domains and try out a maximum of 3 domains of your choice. You are then required to undergo and clear the assignments based on your selection and qualify the Level 2 assignments after which you shall be taken forward for Project Based Learning under Summer Internship 2023. The assignments are to be attempted and cleared latest by 28th May 2023. Internship Projects: After successful completion of Level-2, Interns will be allocated projects based on domain preference, level-2 performance and interaction with the mentors. Final Selection and Project Allocation based on the Level 2 performance and preferences These are full-time internships and require 48 hours per week .01Augmented Reality  AR-based fault diagnosis of RWC Implementing AR for complete Demonstration of Robotic Welding Cell Unity ROS Cobot FSM AMR AR FSM Interactive lab introduction . 02Digital Twin  Digital Twin of Robotic Welding Cell for process optimization and Predictive Maintenance. Digital twin for Advanced Milling Machine Centre Industrial Automation CP Lab Autonomy Phase - 2 Multi Process Robotic Cell Simulation on a simulation platform Robotic Welding Cell Simulation on a simulation platform Training Course on Rockwell Kit Training Course on Siemens Kit 04Industrial IoT Implementing IIoT on CPL Autonomy Implementing IIoT for Plant Owner on Robotic Welding Cell  05Machine Learning Remaining Usable Life Estimation (NASA Turbine Dataset) Power Line Fault Detection Computer Vision to detect defects in PCB Power Line Fault Detection Steel Defect Detection Using Computer Vision 06Manufacturing Execution System  Digital quality report generation for 3d prints. Admin & customer pages for Order Management Portal (RWC) 07Real Time Dashboard Visualization of Multi-Process Robotic Cell Interactive Portal for RWC: Supervisor 08Robotics  MPRC OLP Generation using Sim4.1 for a product Use of AMR-Cobot to place pallet ROS Integration of AMR Integration of computer vision with Cobot Computer Vision Integration with Cobot. Here are some videos of our interns presenting their final projects in front of the jury members and their mentors. Jury members were invited from different industries with their expertise in respective domains. In the presentation, our interns explained their whole journey and gave the detailing of the projects. After reviewing all these projects, the top 10 interns got awarded the best project award in their domain and a cash prize of 20b95000. Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in  Short-term education and training courses will be offered along with on-line learning and assessment tools to educators and users on a chargeable basis. All the 12 technology streams will be covered. The domain of Smart Manufacturing is expected to open up a new opportunity in industrial consulting, especially with regard to identification and removal of bottlenecks in current operations. The CEFC multi-disciplinary teams will undertake feasibility surveys, ROI analysis and process improvement studies for clients, prior to implementation and also post implementation.  Many times clients wish to test out the Proof-of-Concept on their actual production lines. The CEFC team assigned to the Client during concept formulation will undertake such site activities with a pool of industry solution providers and system integrators. It is expected that Skill Councils such as Automotive Skill Council, Capital Goods Skill Council,Industrial Automation Skill Council and Tool Rooms will be creating their unique set of Role based competencies. The CEFC will offer them and their Training Partners, a gap filling option for theory and practical, and a brand-agnostic independent certification.  The research team will help users to bring uniqueness into their solution while providing adequate safeguards for IP protection. Users can sponsor prototyping or R&D projects as per their need. IAFSM will take up research projects in consultation with industry for development of cyber physical systems for machine tools, industrial IoT, machine data cloud, machine controllers etc. to bridge the technology adoption gap in manufacturing. These research projects will be executed with clear objectives to meet focussed technology requirement for smart manufacturing and easy adoption by the industries. The developed technologies will be licensed further for commercialization to interested industries.  The CPS facility and the associated technologies will provide a one stop shop for users whose needs cut across multiple disciplines and who want to witness a comprehensive digital transformation before taking up for actual implementation. This will enable clients to experiment and innovate with an appropriate mix of standard and customised solutions. The testing service will enable clients to test and debug problems before installing the solution at site.  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Naman Kapoor Rajesh Bhatt Arun Kumar Vaibhav Anand Atul Tahiliani Amit Sanjeev Kumar Pulak Rijhwani July 2022 - January 2023  S. Vishnukkumar November 2019 - September 2022 Ananya Markande February 2022 - August 2022 Dheeraj R February 2022 - August 2022 Chandrashekhara K L February 2022 - August 2022 Ronak Bhanushali August 2021 - July 2022 Deepak Gattani January 2022 - July 2022 Janhavi Namjoshi August 2021 - February 2022 Khusboo Singh September 2020 - December 2021 Parth Sharma January 2019 - October 2021 Devesh Tarasia September 2020 - August 2021 Venkata Akilesh Manchikanti September 2020 - August 2021 Akash Pardasani February 2021 - August 2021 Dhruv Talwar March 2021 - August 2021 Preeti Joshi August 2018 - June 2021 Prakhar Mishra June 2018 - December 2020 Ishaan Kohli July 2019 - August 2020 Gopi Krishna Bali November 2019 - April 2020 Ashwani Mahajan April 2018 - September 2018 Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in  PTC Experts Visit 12th June, 2023  PTC representatives, Mr. Carmelo Giulio Cinardi and Mr. Vinod Palakkil visited the IITD-AIA Foundation For Smart Manufacturing. The FSM team warmly welcomed them and presented technology demonstrations, highlighting the facility's advancements in smart manufacturing and industry 4.0 with PTC's support. It was a valuable opportunity to showcase innovation and collaboration in this field.  Industry Experts Visit 28th April, 2023 IITD-AIA Foundation For Smart Manufacturing had the immense pleasure of welcoming different industry experts at our facility on Friday 28th April, 2023 and had a discussion over key technologies of smart manufacturing in the domain of industry4 with some technology demonstrations of IIOT, AR, ML etc. at our cyberphysical lab by our experts. Here are some of the glimpse of the day.  Pimpri Chinchwad College Of Engineering Visit10th April-2023 It was an honor for IITD-AIA Foundation For Smart Manufacturing to host one of the leading engineering college Pimpri Chinchwad Education Trust's. Pimpri Chinchwad College Of Engineering at our facility in Indian Institute of Technology, Delhi on April 10, 2023. We had the privilege of showcasing our key technology demonstration in the smart manufacturing domain. which was a truly enriching experience for all involved. We are eager to continue fostering such valuable opportunities for collaboration and growth in the future.Here are some of the glimpse of the day. Roto Pumps Visit3rd April-2023 It was an honor for IITD-AIA Foundation For Smart Manufacturing to host one of the leading industry leader Roto Pumps Ltd at our facility in Indian Institute of Technology, Delhi on April 3, 2023. We had the privilege of showcasing our key technology demonstration in the smart manufacturing domain, which was a truly enriching experience for all involved. We are eager to continue fostering such valuable opportunities for collaboration and growth in the future. Here are some of the glimpse of the day. Charg  2019Affaires Ambassador of USA visited on 23th March-2023 It was an absolute honour to host Elizabeth Jones (Charg  d  2019Affaires Ambassador of USA), Gloria Berbena (Public Diplomacy Minister Counselor), Drew Schufletowski (EEST Minister Counselor), Ann Mason (EEST), Noopur Singh (EEST Site Officer) at IITD-AIA Foundation For Smart Manufacturing facility in Indian Institute of Technology, Delhi. This is part of DST-NSF Joint Research projects on \Secure and Resilient Control of Robotic Systems in CyberPhysical Assembly Line\ where Prof. Sunil Jha is working with Prof. Quanyan Zhu, NYU Tandon School of Engineering. Cybersecurity issues in smart connected factory explained and also direction to make them more resilient. We would like to Thank IHFC (TIH of IIT Delhi) Project Director Prof. Subir Kumar Saha, IHFC CEO Ashutosh Dutt Sharma for giving us this opportunity. Also thanks Narender Singh, Susha Lekshmi.S.U, PhD from IHFC for the support. Different Industry experts Visit20th March-2023 IITD-AIA Foundation For Smart Manufacturing had the immense pleasure of welcoming different industry experts at our facility and had a discussion over keytechnologies of smart manufacturing in the domain of industry4 with some technology demonstrations of IIOT, AR, ML etc. at our cyberphysical lab by our experts.Here are some of the glimpse of the day. Foreign Delegates from B20 India Task Force Visit19th March-2023 IITD-AIA Foundation For Smart Manufacturing had an opportunity to welcome foreign delegates from B20 India Task Force at our facility in Indian Institute of Technology, Delhi. It was lively discussion about digitaltransformation and understanding the keytechnologies of industry4. We are thankful to Confederation of Indian Industry for organizing this visit.Here are some of the glimpse of the day. Anil Sasidharan, Mitsuru Tanabe, Cheng Zhang, Hitomi Kamata from Sony Visit15th March-2023 IITD-AIA Foundation For Smart Manufacturing welcomed Anil Sasidharan, Mitsuru Tanabe, Cheng Zhang, Hitomi Kamata from Sony at our cyber-physical facility with some Technology demonstrations by our experts.Here are some glimpses captured during the visit. Dr. Gloryswarupa Sunchu Director General at NI-MSME visit1st March-2023 IITD-AIA Foundation For Smart Manufacturing welcomed Dr. Gloryswarupa Sunchu (Director General at National Institute for Micro, Small and Medium Enterprises (ni-msme) ) and Mr. Gopala Naidu at our cyber-physical facility with some technology demonstrations by our experts .Here are some glimpses captured during the visit. Mr. Jose Korath, a distinguished industry expert from Tata Steel Jamshedpur visit17th February-2023 IITD-AIA Foundation For Smart Manufacturing had the privilege of hosting Mr. Jose Korath, a distinguished industry expert from Tata Steel Jamshedpur, at our facility today. His insights on the latest developments in the industry were both informative and enlightening. It was an honor to learn from his extensive experience and expertiseHere are some glimpses captured during the visit. Mr. Alan Donnelly (Executive Chairman) and Ms. Hatice K  00fc  00e7  00fck Beton (Chief Executive Officer) from Sovereign Strategy visit15th February-2023 We are fortunate to share that Mr. Alan Donnelly (Executive Chairman) and Ms. Hatice K  00fc  00e7  00fck Beton (Chief Executive Officer) from Sovereign Strategy visited our facility along with Mr. Paramjit Singh and Mr. Siddhant Bhardwaj. It was a great opportunity to exchange ideas and learn from their insights.Here are some glimpses captured during the visit. Mr.Jagdish Prasad ( Vice President) from MTC Group visit15th February-2023 IITD-AIA Foundation For Smart Manufacturing is happy to share that Mr. Jagdish Prasad ( Vice President) from MTC Group which is one of the leading scrap processing companies in India, today he visited our facility for understanding the key technologies of Industry 4.0 so that they can implement automation in the scrap processing and make this process more smart and sustainable for recycling of waste material. It will be a great opportunity for us to work with the MTC group. Looking forward to exploring the difficulties of the industry and providing solutions.Here are some glimpses captured during the visit. Project Review Management Committee visit30th January - 2023 IITD-AIA Foundation For Smart Manufacturing is honored to welcome Project Review Management Committee Mr. Purnendu Sinha, PhD, FIE., Mr. A.N. CHANDRAMOULI (Mouli), Mr. Vijay Kalra, and Mr. Sahil Bhagwanani. We are thankful to the committee members for their recommendations and suggestions on various projects for the welfare of the Indian Industries.Here are some glimpses captured during the visit. Honorable Secretary - Ministry of Heavy Industries, Govt. of India Shri Kamran Rizvi visited25th January - 2023 IITD-AIA Foundation For Smart Manufacturing is proud to share that Honorable Secretary - Ministry of Heavy Industries, Govt. of India Shri Kamran Rizvi visited our facility with his esteemed colleagues : Shri Vijay Mittal (Joint Secretary - MHI), and Shri Vikas Dogra, (Director - MHI) on 25th January, 2023. We are grateful to them for their valuable guidance & support towards the creation of Smart Manufacturing ecosystem for Indian Industries. Such interactions motivate us to perform better everyday.Here are some glimpses captured during the visit. ICF Certified Executive CEO Coach visit 13th January - 2023 We are delighted to share that on Friday, 13th January, 2022, delegated guests Dr. Praveen C Srivastava ICF Certified Executive CEO Coach I Author visited IITD-AIA Foundation For Smart Manufacturing Research Centre. We are thankful to him for sharing his views with us.Here are some glimpses captured during the visit.  Rockwell Automation and Tata Consultancy Services visit 6th January - 2023We are delighted to share that on Friday, 6th January, 2022, delegated guests from Rockwell Automation and Tata Consultancy Services visited IITD-AIA Foundation For Smart Manufacturing Research Centre. We are extremely thankful to Mr. Prasad Penkar, Mr. Dipanjan Banerjee for sharing their views with us.Here are some glimpses captured during the visit.  Pro MFG Media visit 8th December - 2022 We are delighted to share that yesterday on 8th December, 2022, delegated guests from Pro MFG Media visited IITD-AIA Foundation For Smart Manufacturing Research Centre. We are extremely thankful to Pro MFG Media team: Mr. Manish Kulkarni, Mr. Saikumar Shanmugam, Mr. Niranjan Mudholkar.Here are some glimpses captured during the visit. KPMG visit 24th November - 2022 We are delighted to share that today on 24th Nov 2022, delegated guests from KPMG visited IITD-AIA Foundation For Smart Manufacturing Research Centre. We are extremely thankful to the KPMG team : Mr. Sandeep Chittora, Mr. Saurabh Bhatnagar, Mr. Ankit Mudgal, Mr. Palash Sharma  IHFC (TIH of IIT Delhi) visit18th November - 2022 In partnership with IHFC (TIH of IIT Delhi), IITD-AIA Foundation For Smart Manufacturing organized a workshop on Industry 4.0 on November 18th, 2022 with a focus on its significance and impact, IHFC Grand Projects, Smart Manufacturing Technology Demonstration, and R&D opportunities in Industry 4.0.Prof. Sunil Jha, Prof. Amit Kumar Jain, and Prof. Manan Suri conducted interactive sessions with the audience during the Grand Project Talk on Industry 4.0.  Janatics India Private Ltd. visit 14th November - 2022 On 14th Nov 2022, delegated guests, Mr. Sandeep Gupta, Mr. Aravind Ramachandran & Mr. S Sunil Kumar from JANATICS INDIA PRIVATE LTD. visited FSM Research Centre at IIT Delhi. Here are a few glimpses of the visit.  ArcelorMittal Nippon Steel India visit 3rd November - 2022 We are delighted to share that yesterday on 3rd Nov 2022, delegated guests from ArcelorMittal Nippon Steel India visited IITD-AIA Foundation For Smart Manufacturing Research Centre. We are extremely thankful to AM/NS team: Arunkumar Pillai, Nandini Dasgupta, and RAVI S R.Here are some glimpses captured during the visit.  Shri Dr. Rajagopala Chidambaram-Chairman-BoG, IIT Delhi, visit 10th October - 2022 IITD-AIA Foundation For Smart Manufacturing is delighted to have Dr. Rajagopala Chidambaram-Chairman-BoG, Indian Institute of Technology, Delhi, Prof. Rangan Banerjee-Director IITD, Prof. M.R Ravi-HOD-Mechanical Department, IITD. Thank you so much sir for investing your precious time in visiting FSM and sharing your valuable feedback with us.  Dr. SK Gupta, visit 22th September - 2022 It is a great pleasure to share glimpses of the very informative and educative talk with Satyandra Gupta from University of Southern California on developing human-centered automation using Robotics and Machine Learning at Foundation for Smart Manufacturing Research Centre, IIT Delhi.  Hero moto corp visit  4th August - 2022A group of Hero moto corp delegates visited FSM on 4th Aug , 2022. Prof. Sunil Jha, addressed the attendees and discussed the Smart Solutions for Manufacturing Industries while focusing on the need of Skilling and Educational Trainings in Manufacturing industries for professionals. Traditionally, manufacturing has been considered as a process that convert raw materials into physical products in the factories by managing resources with best automation practices available. The digital factory concept offers an integrated approach to enhance the product and production engineering processes and simulation is a key technology within this concept.  Until now, industrial robots always worked separately from humans in specially safeguarded protected spaces. Smart manufacturing has broken down this barrier with a new generation of collaborative industrial robots. A key component in making Industry 4.0 a reality are machines that can produce the desired components faster, more flexibly and more precisely than ever before. Also Consumers want products that reflect their individuality.  Augmented reality is the integration of digital information with the user  ‘s environment in real time. Unlike virtual reality, which creates a totally artificial environment, augmented reality uses the existing environment and overlays new information on top of it. Networked production and control processes in complex machine environments determine the industrial future and make Industry 4.0 possible in the first place. Smart Sensors already today support dynamic, real-time-optimized, and self-organized industry processes.  Being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. Remote access could also help to better prepare service technicians with the necessary information for their tasks. Knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. The world of automation is merging with the IT world. Safety & security is an important prerequisite for the function of Industry 4.0 systems, which in contrast to traditional production plants have interfaces to their environment.  Indian Institute of Technology Delhi (IITD) and Automation Industry Association (AIA) joined hands in May, 2017 to set up a center under auspices of Samarth Udyog - a project of the Ministry of Heavy Industries (MHI). The center is called Foundation for Smart Manufacturing (FSM), and it helps supports and develops technologies for right understanding and implementation of concepts of smart manufacturing. FSM is a demo-cum-experience facility in North India, vests in extensive skill building, MSME consultancy, multi-academia partnerships and research which will give a huge fillip to technology aided competitiveness of Indian manufacturing. The project will imbibe technologies from Europe, Japan, USA and India. 'Producing Smarter' with Technologies. FSM has been launched specifically for implementation of smart manufacturing in India which it undertakes to do effectively by: Awareness Building, Prototyping, Simulation and testing Services, Consulting Services, Site Integration Services, Education and Training, Skills Certification, Job work and Research. Mandated Guidelines Awareness campaigns on Industry 4.0 Training for master trainers Active participation provisions for start-up / incubators Hand-holding of SMEs to plan and implement relevant Industry 4.0 projects to be done through consultancy services on a chargeable basis Collaborating with neighborhood universities for student training/internship programs Involving industry in SPV members model for sustainability Participating in a Government-formed platform for Industry 4.0 To make adequate provisions for e-waste management Involving as many clusters of capital goods' as possible  At FSM we are committed to demystify the smart manufacturing technologies by learning ourselves and then transfer the learning through various training programs to industry and academicians. We are engaging researchers to indigenously develop technologies to make it affordable by Indian MSME and facilitate adoption of Industry 4.0. Our state of art cyber physical lab showcase technologies available across globe through our strong partnership with the automation industries. Director With a far-sighted approach, the Government of India has supported Automation Industry and IIT Delhi in creation of advanced engineering and software facilities for smart manufacturing. Prospective integrators, consultants and end users can use our training and tryout services to validate their proof-of-concept before scaling up. We help demystify and de-risk your investments in Industry 4.0 Co-Creator Dr. Sunil Jha Director Mr. Dilip Sawhney Director  Dr. Anil Wali Director  Sunil Mehta Director Mr. Anup Wadhwa Director Automation Industry Association Mr. Ravi Agarwal Director Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Event Date On going Mode Of Learning Virtual IITD-AIA Foundation For Smart Manufacturing presents a comprehensive Learning Module on 201cIndustrial Automation and IoT using soft-PLC  201d Program, visualize, execute remote monitoring, motion programming and more using soft-PLC on the upcoming Learning Module. This program is meant for Engineering Students and Industry Freshers wanting to gain practical learning in the field of Industrial Automation which is foundation for Smart Manufacturing implementation. Learning content prepared under guidance of Prof. Sunil Jha  Event Date16th May -16th July, 2023 Mode Of LearningVirtual IITD-AIA Foundation For Smart Manufacturing is offering an 2 Months Full-time Online summer internship starting from 16th May 2023 to 16th July 2023 in smart manufacturing domains. Last Date for Application: 15th-April-2023. Event Date20th-24th February, 2023 IITD-AIA Foundation For Smart Manufacturing is delighted to announce the successful conclusion of our week long training program titled \Competency Building in Industry 4.0\ for industry experts from Hindustan Aeronautics Limited (HAL) in the field of smart manufacturing  . HAL has entrusted us with the task of upskilling their experts from various domains. Our carefully crafted learning sessions provided valuable insights into the major domains of Industry 4.0. Including Automation, IIoT, Augmented Reality, Robotics, AI, ML  Event Date15th February, 2023 IITD-AIA Foundation For Smart Manufacturing got an opportunity to be a part of G20 Digital Economy Working Group (DEWG) in Lucknow hosted by UP Government. These type of events helps all the segments of nation like Politicians, Economic dignitaries, Business leader, Entrepreneurs, Industry Associates etc. to come on a common platform and take nation towards a bright and better future and we are proud to be participating and showcasing live demonstrations of our work to the visitors. Event Date22th December, 2022 We are happy to inform that on December 22, 2022, IITD-AIA Foundation For Smart Manufacturing presented various industry 4.0 technologies at Workshop on Automation for Smart Manufacturing organized by Automation Industry Association in Pune. Visitors were given Live Demonstrations of technologies created for smart manufacturing in the cyber-physical lab at the Indian Institute of Technology, Delhi. Here are some of the day's highlights.  Event Date10th December, 2022 We are happy to report that on December 10, 2022, IITD-AIA Foundation For Smart Manufacturing presented various industry 4.0 technologies at industry day at Indian Institute of Technology, Delhi. Live Demonstrations of Technologies developed in Smart Manufacturing were given to the visitors. Here are some of the day's highlights. Event Date29th November, 2022 Mode Of Learning Offline (4PM to 6PM) IITD-AIA Foundation For Smart Manufacturing, takes immense pleasure to invite Engineering Students & Fresh Graduates for a Workshop on Industry 4.0 on the occasion of its 5th Foundation Day. Join us for an interactive session with FSM experts on Industry 4.0 & Career Opportunities in Smart Manufacturing domain.Date : 29th November, 2022Timings : 4PM to 6PMVenue : Seminar Room, Room no. 422-B,Mechanical Engineering Department, IIT Delhi.  Mode Of Learning Offline IITD-AIA Foundation For Smart Manufacturing presents Graduate Engineer Trainee programme starting from 1st September.An excellent opportunity for freshers to step into learning smart manufacturing. Benefits:Gain the knowledge about Smart Manufacturing & Automation Technology.Get hands-on experience on a host of cutting edge Industrial Automation equipment.Learn device interfacing, communication, cloud engagement, data dashboarding techniques and much more.Build your career to the next level.Get the opportunity to learn under the guidance of Prof. Sunil Jha! Event Date13th October, 2022 UPSKILLING IS THE NEED OF THE HOURSKILLING OPPORTUNITIES FOR SMART MANUFACTURING REVOLUTIONJOIN US ON OCTOBER 13TH, 11AMRegistration link:https://lnkd.in/dGAk89nwGet to know about New Age Skills that you can acquire and enhance your career success.Join the discussion and get aligned with the future Explore the Smart Manufacturing Revolution with us.  Event Date7th-October We are delighted to share that on 7th Oct 2022, IITD-AIA Foundation For Smart Manufacturing showcased I4.0 Technologies to Honorable Minister (MHI)-Dr. Mahendra Nath Pandey, MoS (HI)-Shri Krishn Pal Gurjar, at National conference conducted by MHI on Industry 4.0 at Kevadia, Gujarat. During the conference, team FSM demonstrated Self Guided Maintenance on the Smart Training Kit using Augmented Reality, Remote Monitoring and Control of the CP Lab at Indian Institute of Technology, Delhi via IIOT where the visitors were able to place order and monitor the progress on the Live Dashboard. Event Date30th July, 2022 IITD-AIA Foundation For Smart Manufacturing along with Indian Institute of Technology, Delhi participated in the \Academic Outreach Program\ held at the IIT-Delhi Sonipat campus. All the participating academic and skilling institutions in this program expressed their appreciation for showcasing the latest smart manufacturing and automation technologies. FSM initiatives for creating a platform for exploring and implementing smart manufacturing concepts and practices in India were very well received by students, academic and industrial institutions. The event focused on the sharing of knowledge for solving problems in manufacturing through automation and smart manufacturing technologies thereby helping to increase the competitiveness of the domestic manufacturing sector.  Event Date1st June-31st July, 2022 Mode Of Learning Virtual / Physical IITD-AIA Foundation For Smart Manufacturing is offering 100 openings for 2-Months Full-time Online Summer Internship starting from 1st June 2022 to 31st July 2022 under the theme of 201cSmart Factory  201d.. Event Date9th-13th March, 2022 Mode Of Learning Virtual IITD-AIA Foundation For Smart Manufacturing presents a 5 Days/10 Hours Online Self Learning Program on components and integration for Smart Manufacturing. This program is meant for Engineering Students and Industry Freshers wanting to gain practical learning in the field of Industrial Automation which is foundation for Smart Manufacturing implementation.Learning content prepared under guidance of Prof. Sunil Jha  Learning content prepared under guidance of Prof. Sunil Jha IITD-AIA Foundation For Smart Manufacturing presents 4 hours free awareness course on Industrial Automation Learning. This is a supplement course helpful for the upcoming 5 Days/10 Hours Online Self Learning Program on components and integration for Smart Manufacturing  The 2nd International Conference was held on 11-12 Jan 2022 at the Indian Institute of Science, Bangalore, India.6 interns from FSM participated and presented research papers at the conference and 4 Research papers were submitted by the FSM interns.The papers 201c*Augmented Reality Implementation for Fault Diagnosis on Robotic Welding Cell  201d and *  201cAutonomous Navigation for Mobile Robots with Sensor Fusion Technology  201d** were awarded the Distinguished Papers at the conference On Oct 21, 2021, FSM and MathWorks hosted an online event to bring together experts from Industry and Academic to showcase technology and workflows that aimed to facilitate Fourth Industrial Revolution and generate value for your digital transformation.  During the months of June-July 2021, FSM organized a Summer Online Internship. 900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns. The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as 201cAugmented Reality  201d, 201cMachine Learning  201d, 201cAutomation  201d, and 201cIIot  201d. The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Multi Process Robotic Cell Smart Lathe Machine Robotic Welding Cell FSM AR Demo Mechanism Kit Assembly and Disassembly AR Maintenance Using HoloLens Expert Maintenance System Using HoloLens Cyber Physical Assembly Line CP Lab Autonomy Smart Mechanism Kit Realtime Values PLC Kit Parts Introduction  Smart Mechanism Kit Motor Replacement Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in  The Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners.  We envision integrated technology advancement in the Indian Manufacturing Sector to unleash the hidden potential of small and medium enterprises. We aim to achieve this by carrying out a holistic intervention called Smart Manufacturing; that imbibes innovative approaches around which smart technology enabled manufacturing (STEM)can fructify. Many aspects of Industry 4.0, Industrial Internet Consortium and e-Factory can be broken down into 201cautomated practices  201d that can incrementally be connected into a manageable and logical process to suit short-term needs of investors. This practical and cost-effective approach would also generate a natural momentum toward long-term goals. The national Common Engineering Facility Center will act as the medium to accomplish this vision of developed India by integrating the research expertise of IITD and invaluable practical experience of industry. On assessing the exciting advantages as well as various issues we face in adopting Industry 4.0, there is one overriding conclusion that we all need to acknowledge the dream of wanting to somehow just jump in and excitedly resolve the entire implementation challenge all at once.  For one thing, requirements for Industry 4.0 are just too comprehensive and unwieldy to consider tackling it all at once. For another, Industry 4.0 will likely take years to be commonplace. But in nutshell, Industry 4.0 will never be 201ccomplete  201d anyway, it is a continuous-improvement tool, flexibly changing and expanding its iterations as the world evolves. The Smart Manufacturing initiative will revolutionize the production operations in SMEs by improving their quality, reducing throughput time and inventory cost. New analytics solutions help our Indian customers move ahead on their Smart Manufacturing journey, no matter where they are today. This will lead to a simplified approach to analytics converting data into decisions. This will also enable flexibility to the manufacturers to customize their products to cater the high expectations of their consumers. This will also improve the quality of jobs and uplift the standard of living of people.  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Short term education training offered along with online learning and assessment tool. Participants are engaged for approximately 100 hours in different technologies that include Automation, Industrial IoT, Augmented Reality, Robotics, Digital Twin and etc. The learnings are divided into Awareness Level, Learning Level and Expert Level. Smart Manufacturing Training Kits Software CPS facilities and the associated technologies will provide for one-stop shop for users whose needs are cut across multiple disciplines and who wants to witness comprehensive digital transformation before taking up the actual implementation. Cyber Physical Assembly Line Multi Process Robotic Cell Robotic Welding Cell Mobile Collaborative Robot  Cyber Physical Factory We are providing following services: 3D printing Smart Manufacturing Training Kits for Industrial Automation learning MIG Welding (including Wire-arc-additive-manufacturing)  Multimaterial 3D printer Smart Manufacturing Training Kits MIG Welding Cell Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in  The Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners.  We envision integrated technology advancement in the Indian Manufacturing Sector to unleash the hidden potential of small and medium enterprises. We aim to achieve this by carrying out a holistic intervention called Smart Manufacturing; that imbibes innovative approaches around which smart technology enabled manufacturing (STEM)can fructify. Many aspects of Industry 4.0, Industrial Internet Consortium and e-Factory can be broken down into 201cautomated practices  201d that can incrementally be connected into a manageable and logical process to suit short-term needs of investors. This practical and cost-effective approach would also generate a natural momentum toward long-term goals. The national Common Engineering Facility Center will act as the medium to accomplish this vision of developed India by integrating the research expertise of IITD and invaluable practical experience of industry. On assessing the exciting advantages as well as various issues we face in adopting Industry 4.0, there is one overriding conclusion that we all need to acknowledge the dream of wanting to somehow just jump in and excitedly resolve the entire implementation challenge all at once.  For one thing, requirements for Industry 4.0 are just too comprehensive and unwieldy to consider tackling it all at once. For another, Industry 4.0 will likely take years to be commonplace. But in nutshell, Industry 4.0 will never be 201ccomplete  201d anyway, it is a continuous-improvement tool, flexibly changing and expanding its iterations as the world evolves. The Smart Manufacturing initiative will revolutionize the production operations in SMEs by improving their quality, reducing throughput time and inventory cost. New analytics solutions help our Indian customers move ahead on their Smart Manufacturing journey, no matter where they are today. This will lead to a simplified approach to analytics converting data into decisions. This will also enable flexibility to the manufacturers to customize their products to cater the high expectations of their consumers. This will also improve the quality of jobs and uplift the standard of living of people.  IITD-AIA Foundation For Smart Manufacturing presents a comprehensive Learning Module on 201cIndustrial Automation and IoT using soft-PLC  201d Program, visualize, execute remote monitoring, motion programming and more using soft-PLC on the upcoming Learning Module. This program is meant for Engineering Students and Industry Freshers wanting to gain practical learning in the field of Industrial Automation which is foundation for Smart Manufacturing implementation.Learning content prepared under guidance of Prof. Sunil Jha  IITD-AIA Foundation For Smart Manufacturing presents 4 hours free awareness course on Industrial Automation Learning. This is a supplement course helpful for the upcoming 5 Days/10 Hours Online Self Learning Program on components and integration for Smart Manufacturing Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in  Short-term education and training courses will be offered along with on-line learning and assessment tools to educators and users on a chargeable basis..  Fill out the online registration form Shortlisting based on the eligibility criteria Level 1: Learning modules (Approx. time: 24 hrs) will be made available on fsmskills website. To be completed by 20th May 2023. Post your signup on fsmskill, you will gain access to the Level-1 course of IAFSM. This step involves learning of Smart Manufacturing Concepts and is the foundation course that is mandatory for all interns to qualify latest by 20th May 2023. You are required to go through recorded lectures and study the content thoroughly to clear the assignments of Level-1 in maximum 2 attempts. After 2 attempts, if you are not able to qualify for Level-1, your candidature will no longer be considered for internship. Level 2 : After successful completion of Level-1, Indian Institute of Technology Delhi (IITD) and Automation Industry Association (AIA) joined hands in May, 2017 to set up a center under auspices of Samarth Udyog - a project of the Ministry of Heavy Industries (MHI). The center is called Foundation for Smart Manufacturing (FSM), and it helps supports and develops technologies for right understanding and implementation of concepts of smart manufacturing. FSM is a demo-cum-experience facility in North India, vests in extensive skill building, MSME consultancy, multi-academia partnerships and research which will give a huge fillip to technology aided competitiveness of Indian manufacturing. The project will imbibe technologies from Europe, Japan, USA and India. 'Producing Smarter' with Technologies. FSM has been launched specifically for implementation of smart manufacturing in India which it undertakes to do effectively by: Awareness Building, Prototyping, Simulation and testing Services, Consulting Services, Site Integration Services, Education and Training, Skills Certification, Job work and Research. Mandated Guidelines Awareness campaigns on Industry 4.0 Training for master trainers Active participation provisions for start-up / incubators Hand-holding of SMEs to plan and implement relevant Industry 4.0 projects to be done through consultancy services on a chargeable basis Collaborating with neighborhood universities for student training/internship programs Involving industry in SPV members model for sustainability Participating in a Government-formed platform for Industry 4.0 To make adequate provisions for e-waste management Involving as many clusters of 'capital goods'  as possible At FSM we are committed to demystify the smart manufacturing technologies by learning ourselves and then transfer the learning through various training programs to industry and academicians. We are engaging researchers to indigenously develop technologies to make it affordable by Indian MSME and facilitate adoption of Industry 4.0. Our state of art cyber physical lab showcase technologies available across globe through our strong partnership with the automation industries. Director With a far-sighted approach, the Government of India has supported Automation Industry and IIT Delhi in creation of advanced engineering and software facilities for smart manufacturing. Prospective integrators, consultants and end users can use our training and tryout services to validate their proof-of-concept before scaling up. We help demystify and de-risk your investments in Industry 4.0 Co-Creator Dr. Sunil JhaDirector Mr. Dilip SawhneyDirector  Dr. Anil WaliDirector  Sunil MehtaDirector Mr. Anup WadhwaDirector Automation Industry Association Mr. Ravi AgarwalDirector Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in An augmented reality application to provide self-guided training on the FSM PLC Kits. The user can scan the ThingMark on the Kit to load up the experience or can FSM PLC Kit Parts Introduction 2 use model target detection by aligning the on-screen guide with the real hardware and then click on the parts to know about them Augmented RealityIIoT 01Self-Guided Training 02Action Latency Reduction 03Remote Maintenance 04Self-Guided Maintenance Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in This course is designed on a Simulation platform. It  2019s a self-paced online course where you get to learn different domains of Automation like Pneumatic, Electro-Pneumatics, PLC programming and Electrical Connections of a PLC. You will create different industrial projects and simulate them on the software during the course tenure. This course is a blend of lecture videos, Step-by-step documents, and Live sessions to give a complete idea and knowledge of Industrial Automation to you. By the end of the course, you will be ready to write a PLC program for machines with DI, DO, AI and AO. You will be able to understand and create pneumatic and electro-pneumatic circuits depending on the industrial application.  Pneumatic Components Introduction, Creating Pneumatic connections on software and creating an Industrial Application Electro-Pneumatic Components Introduction, Creating Pneumatic and electrical connections on software and creating an Industrial Application Hardware identification of PLC , PLC Programming on basic instructions, Timers, Counters, and Digital I/OsProject: Programming a PLC for an Industrial Application PLC programming on compare instructions and Analog I/Os Project: Programming a PLC (with analog Inputs and Outputs) for an Industrial Application Self- project where in the participants will be given an opportunity to try out the use case that they have in mind using the simulation software to simulate and test their application Minimum hardware requirement1. 4GB Ram, i3 processor with windows 8 or above version. 2. Windows is a must. Software doesn  2019t run on macos or linux 1. 4GB Ram, i3 processor with windows 8 or above version. 2. Windows is a must. Software doesn  2019t run on macos or linux PrerequisitesNo prerequisites are required No prerequisites are required  Is this limited to any stream?  Any knowledge of PLC is required before joining the course? Is this limited to any stream?No, Anyone interested in industrial Automation can join the course.Any knowledge of PLC is required before joining the course?No. You will be starting from the scratch but having prior knowledge will help you to complete the course early. No, Anyone interested in industrial Automation can join the course. No. You will be starting from the scratch but having prior knowledge will help you to complete the course early. Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in An augmented reality application to provide self-guided training on the FSM PLC Kits. The user can scan the ThingMark on the Kit to load up the experience or can FSM PLC Kit Parts Introduction 2 use model target detection by aligning the on-screen guide with the real hardware and then click on the parts to know about them Augmented RealityIIoT 01Self-Guided Training Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us Administration Operations Marketing ExecutionOffice Execution Human Resource  Hii Everyone, Myself Amit. I belong to Bahadurgarh from Haryana. I did my schooling from Vijaya sr. sec. school in my hometown, than i did my graduation from MDU Rohtak. After that i did my first job in Indiabulls consumer finance limited as a sales and backend executive. I joined FSM on 16th August,2021. I found very opportunites in FSM with very helpful and skilled staff. I started my professional journey as a customer support executive in 2018. Than in November 2018 I joined IndiaBulls Consumer Finance Limited as a backend executive, where I was among the best caller and best sales executive for several months. Than I joined FSM on 16th August 2021 where I came to know about a whole new different experience and new opportunities. Here I learnt about new marketing skills, time managment, co-ordination with the team, new experiences of administrations and operations.  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Naman Kapoor Rajesh Bhatt Arun Kumar Vaibhav Anand Atul Tahiliani Amit Sanjeev Kumar Pulak RijhwaniJuly 2022 - January 2023  S. VishnukkumarNovember 2019 - September 2022 Ananya MarkandeFebruary 2022 - August 2022 Dheeraj RFebruary 2022 - August 2022 Chandrashekhara K LFebruary 2022 - August 2022 Ronak BhanushaliAugust 2021 - July 2022 Deepak GattaniJanuary 2022 - July 2022 Janhavi NamjoshiAugust 2021 - February 2022 Khusboo SinghSeptember 2020 - December 2021 Parth SharmaJanuary 2019 - October 2021 Devesh TarasiaSeptember 2020 - August 2021 Venkata Akilesh ManchikantiSeptember 2020 - August 2021 Akash PardasaniFebruary 2021 - August 2021 Dhruv TalwarMarch 2021 - August 2021 Preeti JoshiAugust 2018 - June 2021 Prakhar MishraJune 2018 - December 2020 Ishaan KohliJuly 2019 - August 2020 Gopi Krishna BaliNovember 2019 - April 2020 Ashwani MahajanApril 2018 - September 2018 Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Smart sensors are a prerequisite for creating the best possible basis for a future-oriented automation system. This is because the smart factory needs data that can principally only be provided by smart, intelligent and communication-enabled sensors. Communication-enabled means being able to exchange sensor data with a machine controller or a cloud-based application. Thus, for example, sensor parameters are automatically adapted to new production orders within seconds. Or a light barrier detects contamination of its optics and reports this directly to the control center.  Smart Sensors generate and receive data and information which goes beyond traditional switching signals or measured process parameters. They therefore enable substantial increases in efficiency, more flexibility, and better planning security for predictive plant maintenance. Depending on the requirement, Smart Sensors cover up to four dimensions of Smart Sensor technology.  1. Enhanced sensing 2. Efficient communication 3. Diagnostics 4. Smart tasks Just like smart sensors, smart actuators also have intelligence built into them. Some of the smart features possessed by different types of actuators are:  Open standards: Multi-protocol communication interface allow universal operation with diverse, ethernet-based communication protocols Web-based commissioning and diagnostics via integrated web server Comprehensive access to actuator data (e.g. torque or power), via web connector for preventive maintenance or database interfacing and data analysis Inbuilt smart sensors for capturing data Flexible configuration according to requirement Comprehensive condition monitoring for fast identification of critical conditions and comfortable analysis Simple connection to higher-level data systems without additional hardware Workpiece-dependent parameters setting guaranteeing quality results Integrated control system with browser-based operating system allows for information sharing across multi vendor data systems  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in  PTC Experts Visit 12th June, 2023  PTC representatives, Mr. Carmelo Giulio Cinardi and Mr. Vinod Palakkil visited the IITD-AIA Foundation For Smart Manufacturing. The FSM team warmly welcomed them and presented technology demonstrations, highlighting the facility's advancements in smart manufacturing and industry 4.0 with PTC's support. It was a valuable opportunity to showcase innovation and collaboration in this field.  Industry Experts Visit 28th April, 2023 IITD-AIA Foundation For Smart Manufacturing had the immense pleasure of welcoming different industry experts at our facility on Friday 28th April, 2023 and had a discussion over keytechnologies of smart manufacturing in the domain of industry4 with some technology demonstrations of IIOT, AR, ML etc. at our cyberphysical lab by our experts. Here are some of the glimpse of the day.  Pimpri Chinchwad College Of Engineering Visit10th April-2023 It was an honor for IITD-AIA Foundation For Smart Manufacturing to host one of the leading engineering college Pimpri Chinchwad Education Trust'S. Pimpri Chinchwad College Of Engineering at our facility in Indian Institute of Technology, Delhi on April 10, 2023. We had the privilege of showcasing our keytechnology demonstration in the smart manufacturing domain. which was a truly enriching experience for all involved. We are eager to continue fostering such valuable opportunities for collaboration and growth in the future.Here are some of the glimpse of the day. Roto Pumps Visit3rd April-2023 It was an honor for IITD-AIA Foundation For Smart Manufacturing to host one of the leading industry leader Roto Pumps Ltd at our facility in Indian Institute of Technology, Delhi on April 3, 2023. We had the privilege of showcasing our keytechnology demonstration in the smart manufacturing domain, which was a truly enriching experience for all involved. We are eager to continue fostering such valuable opportunities for collaboration and growth in the future.Here are some of the glimpse of the day. Charg  d  2019Affaires Ambassador of USA visited on 23th March-2023 It was an absolute honour to host Elizabeth Jones (Charg  d  2019Affaires Ambassador of USA), Gloria Berbena (Public Diplomacy Minister Counselor), Drew Schufletowski (EEST Minister Counselor), Ann Mason (EEST), Noopur Singh (EEST Site Officer) at IITD-AIA Foundation For Smart Manufacturing facility in Indian Institute of Technology, Delhi. This is part of DST-NSF Joint Research projects on \Secure and Resilient Control of Robotic Systems in CyberPhysical Assembly Line\ where Prof. Sunil Jha is working with Prof. Quanyan Zhu, NYU Tandon School of Engineering. Cybersecurity issues in smart connected factory explained and also direction to make them more resilient. We would like to Thank IHFC (TIH of IIT Delhi) Project Director Prof. Subir Kumar Saha, IHFC CEO Ashutosh Dutt Sharma for giving us this opportunity. Also thanks Narender Singh, Susha Lekshmi.S.U, PhD from IHFC for the support. Different Industry experts Visit20th March-2023 IITD-AIA Foundation For Smart Manufacturing had the immense pleasure of welcoming different industry experts at our facility and had a discussion over keytechnologies of smart manufacturing in the domain of industry4 with some technology demonstrations of IIOT, AR, ML etc. at our cyberphysical lab by our experts.Here are some of the glimpse of the day. Foreign Delegates from B20 India Task Force Visit19th March-2023 IITD-AIA Foundation For Smart Manufacturing had an opportunity to welcome foreign delegates from B20 India Task Force at our facility in Indian Institute of Technology, Delhi. It was lively discussion about digitaltransformation and understanding the keytechnologies of industry4. We are thankful to Confederation of Indian Industry for organizing this visit.Here are some of the glimpse of the day. Anil Sasidharan, Mitsuru Tanabe, Cheng Zhang, Hitomi Kamata from Sony Visit15th March-2023 IITD-AIA Foundation For Smart Manufacturing welcomed Anil Sasidharan, Mitsuru Tanabe, Cheng Zhang, Hitomi Kamata from Sony at our cyber-physical facility with some Technology demonstrations by our experts.Here are some glimpses captured during the visit. Dr. Gloryswarupa Sunchu Director General at NI-MSME visit1st March-2023 IITD-AIA Foundation For Smart Manufacturing welcomed Dr. Gloryswarupa Sunchu (Director General at National Institute for Micro, Small and Medium Enterprises (ni-msme) ) and Mr. Gopala Naidu at our cyber-physical facility with some technology demonstrations by our experts .Here are some glimpses captured during the visit. Mr. Jose Korath, a distinguished industry expert from Tata Steel Jamshedpur visit17th February-2023 IITD-AIA Foundation For Smart Manufacturing had the privilege of hosting Mr. Jose Korath, a distinguished industry expert from Tata Steel Jamshedpur, at our facility today. His insights on the latest developments in the industry were both informative and enlightening. It was an honor to learn from his extensive experience and expertiseHere are some glimpses captured during the visit. Mr. Alan Donnelly (Executive Chairman) and Ms. Hatice K  00fc  00e7  00fck Beton (Chief Executive Officer) from Sovereign Strategy visit15th February-2023 We are fortunate to share that Mr. Alan Donnelly (Executive Chairman) and Ms. Hatice K  00fc  00e7  00fck Beton (Chief Executive Officer) from Sovereign Strategy visited our facility along with Mr. Paramjit Singh and Mr. Siddhant Bhardwaj. It was a great opportunity to exchange ideas and learn from their insights.Here are some glimpses captured during the visit. Mr.Jagdish Prasad ( Vice President) from MTC Group visit15th February-2023 IITD-AIA Foundation For Smart Manufacturing is happy to share that Mr. Jagdish Prasad ( Vice President) from MTC Group which is one of the leading scrap processing companies in India, today he visited our facility for understanding the key technologies of Industry 4.0 so that they can implement automation in the scrap processing and make this process more smart and sustainable for recycling of waste material. It will be a great opportunity for us to work with the MTC group. Looking forward to exploring the difficulties of the industry and providing solutions.Here are some glimpses captured during the visit. Project Review Management Committee visit30th January - 2023 IITD-AIA Foundation For Smart Manufacturing is honored to welcome Project Review Management Committee Mr. Purnendu Sinha, PhD, FIE., Mr. A.N. CHANDRAMOULI (Mouli), Mr. Vijay Kalra, and Mr. Sahil Bhagwanani. We are thankful to the committee members for their recommendations and suggestions on various projects for the welfare of the Indian Industries.Here are some glimpses captured during the visit. Honorable Secretary - Ministry of Heavy Industries, Govt. of India Shri Kamran Rizvi visited25th January - 2023 IITD-AIA Foundation For Smart Manufacturing is proud to share that Honorable Secretary - Ministry of Heavy Industries, Govt. of India Shri Kamran Rizvi visited our facility with his esteemed colleagues : Shri Vijay Mittal (Joint Secretary - MHI), and Shri Vikas Dogra, (Director - MHI) on 25th January, 2023. We are grateful to them for their valuable guidance & support towards the creation of Smart Manufacturing ecosystem for Indian Industries. Such interactions motivate us to perform better everyday.Here are some glimpses captured during the visit. ICF Certified Executive CEO Coach visit 13th January - 2023 We are delighted to share that on Friday, 13th January, 2022, delegated guests Dr. Praveen C Srivastava ICF Certified Executive CEO Coach I Author visited IITD-AIA Foundation For Smart Manufacturing Research Centre. We are thankful to him for sharing his views with us.Here are some glimpses captured during the visit.  Rockwell Automation and Tata Consultancy Services visit 6th January - 2023We are delighted to share that on Friday, 6th January, 2022, delegated guests from Rockwell Automation and Tata Consultancy Services visited IITD-AIA Foundation For Smart Manufacturing Research Centre. We are extremely thankful to Mr. Prasad Penkar, Mr. Dipanjan Banerjee for sharing their views with us.Here are some glimpses captured during the visit.  Pro MFG Media visit 8th December - 2022 We are delighted to share that yesterday on 8th December, 2022, delegated guests from Pro MFG Media visited IITD-AIA Foundation For Smart Manufacturing Research Centre. We are extremely thankful to Pro MFG Media team: Mr. Manish Kulkarni, Mr. Saikumar Shanmugam, Mr. Niranjan Mudholkar.Here are some glimpses captured during the visit. KPMG visit 24th November - 2022 We are delighted to share that today on 24th Nov 2022, delegated guests from KPMG visited IITD-AIA Foundation For Smart Manufacturing Research Centre. We are extremely thankful to the KPMG team : Mr. Sandeep Chittora, Mr. Saurabh Bhatnagar, Mr. Ankit Mudgal, Mr. Palash Sharma  IHFC (TIH of IIT Delhi) visit18th November - 2022 In partnership with IHFC (TIH of IIT Delhi), IITD-AIA Foundation For Smart Manufacturing organized a workshop on Industry 4.0 on November 18th, 2022 with a focus on its significance and impact, IHFC Grand Projects, Smart Manufacturing Technology Demonstration, and R&D opportunities in Industry 4.0.Prof. Sunil Jha, Prof. Amit Kumar Jain, and Prof. Manan Suri conducted interactive sessions with the audience during the Grand Project Talk on Industry 4.0.  Janatics India Private Ltd. visit 14th November - 2022 On 14th Nov 2022, delegated guests, Mr. Sandeep Gupta, Mr. Aravind Ramachandran & Mr. S Sunil Kumar from JANATICS INDIA PRIVATE LTD. visited FSM Research Centre at IIT Delhi. Here are a few glimpses of the visit.  ArcelorMittal Nippon Steel India visit 3rd November - 2022 We are delighted to share that yesterday on 3rd Nov 2022, delegated guests from ArcelorMittal Nippon Steel India visited IITD-AIA Foundation For Smart Manufacturing Research Centre. We are extremely thankful to AM/NS team: Arunkumar Pillai, Nandini Dasgupta, and RAVI S R.Here are some glimpses captured during the visit.  Shri Dr. Rajagopala Chidambaram-Chairman-BoG, IIT Delhi, visit 10th October - 2022 IITD-AIA Foundation For Smart Manufacturing is delighted to have Dr. Rajagopala Chidambaram-Chairman-BoG, Indian Institute of Technology, Delhi, Prof. Rangan Banerjee-Director IITD, Prof. M.R Ravi-HOD-Mechanical Department, IITD. Thank you so much sir for investing your precious time in visiting FSM and sharing your valuable feedback with us.  Dr. SK Gupta, visit 22th September - 2022 It is a great pleasure to share glimpses of the very informative and educative talk with Satyandra Gupta from University of Southern California on developing human-centered automation using Robotics and Machine Learning at Foundation for Smart Manufacturing Research Centre, IIT Delhi.  Hero moto corp visit  4th August - 2022A group of Hero moto corp delegates visited FSM on 4th Aug , 2022. Prof. Sunil Jha, addressed the attendees and discussed the Smart Solutions for Manufacturing Industries while focusing on the need of Skilling and Educational Trainings in Manufacturing industries for professionals.  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in This course is completely designed on the Codesys platform using Soft-PLC. Throughout this self-paced online course, participants will be trained on the basics of ladder logic programming, Visualization of a Human-Machine Interface for controlling the program parameters, and Motion programming with a basic linear axis motion to rotary of single-axis motion. This course also explains the communication between the edge tier & platform tier using a data aggregator software, an interface that is created between the Soft-PLC and the data aggregator using OPC-UA Server and Client plug-in. By the end of the course, you will be ready for programming a codesys platform-based hardware with HMI features, programming motion controller with Open PLC blocks, knowledge of how OPC-UA Server-Client works and how to communicate the data of a Soft-PLC on Codesys Driver.  Basics of codesys software and the basic instructions used in the ladder logic programming. Designing a front-end UI for the operator to operate the parameters/tags of the program in the HMI. Overview of function block programming, explaining how the single-axis linear system and the single-axis rotary index system with multiple stations. Extracting the data out of the PLC to control and monitor the application remotely within the network. Self- project where in the participants will be given an opportunity to try out the use case that they have in mind using the Soft-PLC along with HMI screens and remote monitoring. Minimum hardware requirement1. 4GB Ram, i3 processor with windows 8 or above version. 2. Windows is a must. Software doesn  2019t run on macos or linux 1. 4GB Ram, i3 processor with windows 8 or above version. 2. Windows is a must. Software doesn  2019t run on macos or linux PrerequisitesNo prerequisites are required No prerequisites are required  Laptop/ PC minimum requirements  I don  ‘t know what PLC is should I join this course?  Is this limited to any Stream? Laptop/ PC minimum requirements?4GB Ram, i3 processor with windows 8 or above version.I don't know what PLC is should I join this course?Yes, you will get a chance to learn more about it.Where can I find The Economist?Its not limited, anyone with Automation interest can join this course. 4GB Ram, i3 processor with windows 8 or above version. Yes, you will get a chance to learn more about it. Its not limited, anyone with Automation interest can join this course. Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Augmented RealityIIoTCC++PythonJavaReact JSMobile Application DevelopmentFirebaseMachine LearningUnity 3DMRTKVuforia StudioMicrosoft HoloLens 2Mixed Reality  Hi, Vaibhav here. I completed my post-graduation in the Master of Computer Applications from CCS University in 2020. Before joining FSM, I used to teach programming languages. I began teaching with C language and eventually advanced on to teaching C++, Java (SE) and Android application development. I taught for 5 years and completed my post-graduation side by side. At FSM I develop Augmented Reality & Mixed Reality applications using Unity, Vuforia, & Microsoft HoloLens 2. One of my applications, designed for FSM, is also published on the Google Play Store. I have expertise in Augmented Reality applications and work on Python backend and Machine Learning Projects. At FSM I also work as a trainer and have designed the course curriculum for all-rounded hands-on learning, recorded lectures and took lab sessions and guided the students, professors, and employees during the live lab sessions. At FSM I also got the opportunity to work with surgeons from a government hospital to research & develop Augmented Reality applications to be used to train new surgeons and also to help surgeons while doing surgeries. I like learning about all the new tech and adding screens to my desk.  Projects AR Maintenance using HoloLens Expert Maintenance System using HoloLens Smart Mechanism Kit Motor Replacement Sequence using HoloLens FSM AR Demo Unity ROS Cobot using HoloLens AR Surgery Bent Plate AR Surgery Spine Screw Insertion Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in This Multi-purpose Robotic Cell is assembling different variants of direction control valves. It has different stations which performs different operations on the valve body. Assembly,Vision Inspection,Testing,Packaging are some of the operations that can be performed by this cell. The main feature of this cell is that all the programming is done on PLC, use of teach pendant or robot controller programming has been Bypassed. As we can use this cell for multiple purpose,we call it as Multi-Process Robotic Cell. There are different station in the robotic cell:1.Conveyor station:- Use to transfer the inventory tray or pallet to the place where robot can pick them.2. Assembly Station:- All the parts (Axisymmetric and prismatic parts) will be assembled and screwed on the valve body in this station. Along with that inventory scanning and vision inspection of the valve bodies will be done in this station.3.Testing Station:- All the ports of the valve body will be tested in this station. If there is any leakage, valve body will be rejected.4.Hex nut assembly station:- Hex nut will be placed on the body in this station.5. Packaging station :- Assembled valve bodies will be placed in a box and packaging is done. A printing head will print the required details( Type of bodies ,number of bodies, date of assembly etc  2026) on the box and then the box will be dispatched. 1.Siemens PLC:- Siemens S7 1516 is the master plc in this station and it is responsible for all the programming in the cell.2. DI and DO: - 32-point digital input and 32 point digital output module is connected on the base rack with the plc.3. 6 Axis Kuka robot and one axis:- 6 Axis kuka robot performs all the motion operation in the cell.7th axis rotates the valve body during assembly and inspection.4. QR code scanner:- QR code scanner performs the inventory check as it scans the QR code present on body. After scanning a body, plc gets the data of that particular body and its location on the tray.5. Laser based Vision sensor:- This sensor inspects all the four faces of the body and if it is OK then the station will pass the body to next station or else it will reject the body and picks another body of the same type.6. Tool Changers :- Different slave tool changersare used for different operations. These slave tool changers engages with the master tool changer(Present on the Robot arm) when required. Tool changers in this cell are used for:- QR code scanner,Pneumatic gripper and Screwdriver, Vacuum Gripper Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in The Multi-material Additive Manufacturing Machine is a versatile machine made by Hyrel 3D and AMS. It is not 'just a 3D printer''. It is a Large format, Rapid Prototyping, Research, Manufacturing and Educational Platform. This 3D printer is customizable on various levels: build volume, print bed, extruder, and more. It is equipped with the capacity to attach 5 tool heads simulatneously. We have tool heads for printing common FDM filaments(PLA, ABS, PC, PEEK), clays, biologicals, flexibles, UV-curables, RTV Silicones, etc. It is highly evolutive and also offers options for 4th and 5th axes, PCB milling tool, 3-phase machining tool and CO2 or Diode lasers (laser cutting). Smart Manufacturing Training Kit-> Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Traditionally, manufacturing has been considered as a process that convert raw materials into physical products in the factories by managing resources with best automation practices available. Today, drivers such as technology, sustainability, optimization and the need to meet customer demands have once again encouraged the transformation of the manufacturing industry to become adaptive, fully connected, and aware of its own power quality. One of the most significant trends in manufacturing is of improved information technology solutions involving the union of conventional automation with the information technology. Cyber-physical systems (CPS) are enabling technologies which bring the virtual and physical worlds together to create a truly networked world in which intelligent objects communicate and interact with each other. They are \enabling technologies\ which make multiple innovative applications and processes a reality as the boundaries between the real and virtual worlds disappear. Cyber-physical systems provide the basis the creation of an Internet of Things (IoT) which makes smart services and products possible. A cyber-physical system (CPS) is a 201cthing  201d in the Internet of Things. It is a combination of mechanical, electronic and software components that communicate via a data infrastructure such as the Internet, react flexibly to external influences and exchange data with information systems and other CPSs. In manufacturing facilities, cyber-physical systems will communicate with intelligent, networked industrial production and logistics units-also known as cyber-physical production systems (CPPS). The CPSs exchange information, trigger actions in production and reciprocally control themselves autonomously. This enables industrial processes in manufacturing, engineering, use of materials, supply chain management and life cycle management to be fundamentally restructured and optimized.  Currently, manufacturing data are segmented, detailed and planned for a single scope, stored within the legacy systems, thus preventing the digital continuity that would let use them in the optimal way, independently from where and how they have been collected. Cyber Physical System (CPS) will instead be able to provide the needed information from the physical world while cyber-physical-collaboration environment will enable an efficient analysis, management, sharing and usage of the data and the knowledge elaborated from them and from the experience of involved people. Cyber-physical systems also represent a paradigm break from existing business and market models, as revolutionary new applications, service providers and value chains become possible.  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in CADIIoTIndustrial AutomationPythonApplication Development  I am a passionate engineer that has been a part of the FSM family since August, 21. I have completed my Mechanical Engineering Degree from KJ Somaiya CoE, Mumbai. It has been my great fortune to work under the mentorship of Prof. Sunil Jha. I started my engagement with FSM as a student enrolled in their online certification courses that ran from Apr-Jun 21. The team leveraged IIot & AR technologies to give us interactive real-time access to industrial hardware to deliver an effective and unmatched learning experience. My primary field of study is the domain of 2018Manufacturing Execution System  2019.  Projects Executed in FSM (Engineer) (Joined in August 2021) : MES application based around 3D printer machine (Order Management, Scheduling, Digital Quality Report Generation). Smart Shearing Machine - Conversion of Legacy Machine to Smart Machine Robotic Welding Cell Order Management Portal Mobile Inspection Platform for CP Factory  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in AdministrationOperationsBook KeepingMarketingProcurementPlanningExecution  Hi, I am Naman Kapoor, a former resident of the City of Nawabs. I am a Commerce Graduate from Lucknow University. I have cross functional experience in diverse industries like Life Insurance, Pharmaceuticals, Multi-Level Marketing, Real Estate and Corporate Gifting) in Lucknow & Delhi NCR. I love being in my own company and Music is my favorite pastime. I have been a part of FSM since its inception and I love working for the organization. I have learned a lot about various Smart Manufacturing Technologies and have gained exposure to the Automation Industry. My career started with Bharti Axa LIC Ltd. in 2009 and was primarily responsible for sales generation and spreading awareness of Financial Products. In 2011 I switched to Cipla Ltd. where I handled Respiratory Division (Asthma Segment) and was Promoted within the Span of 4 months to Territory Manager and took over the entire COPD Division and boosted the sales to 54% over the year. In 2013, I switched to MLM Industry as Operations Assistant and was promoted to Zonal Head with responsibility of managing Teams size of 53+ people across UP, MP, Bihar, Jharkhand and Chhattisgarh.  In 2014, I switched to Real Estate and Managed execution and setup of a Restaurant at Lucknow that included Operations Management, Marketing, Branding and Promotions, etc. In 2015 I migrated to Gurugram and joined a Corporate Gifting organization where the prime responsibility was to generate sales. In Feb 2018, I joined IAFSM as Executive-Administration and was promoted to Assistant Manager in August 2018. I am responsible for handling Operations and Administration along with Procurement and Team Lead for all running Projects @ IAFSM. Primarily responsible for communication between the organization and Ministry and play various other roles like Book Keeping, Accounting, Data Management, Event Management and Payroll Handling. Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in  Traditionally, manufacturing has been considered as a process that convert raw materials into physical products in the factories by managing resources with best automation practices available. The digital factory concept offers an integrated approach to enhance the product and production engineering processes and simulation is a key technology within this concept.  Until now, industrial robots always worked separately from humans in specially safeguarded protected spaces. Smart manufacturing has broken down this barrier with a new generation of collaborative industrial robots. A key component in making Industry 4.0 a reality are machines that can produce the desired components faster, more flexibly and more precisely than ever before. Also Consumers want products that reflect their individuality.  Augmented reality is the integration of digital information with the use's environment in real time. Unlike virtual reality, which creates a totally artificial environment, augmented reality uses the existing environment and overlays new information on top of it. Networked production and control processes in complex machine environments determine the industrial future and make Industry 4.0 possible in the first place. Smart Sensors already today support dynamic, real-time-optimized, and self-organized industry processes.  Being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. Remote access could also help to better prepare service technicians with the necessary information for their tasks. Knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. The world of automation is merging with the IT world. Safety & security is an important prerequisite for the function of Industry 4.0 systems, which in contrast to traditional production plants have interfaces to their environment.  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Cyber physical factory (CPF) is also known as Discrete Micro Production Facility or simply Micro Production Facility. It demonstrates the convergence of the Informational Technology (IT) and Operational Technology (OT) at micro factory level where the machines are discretely located. It has 18 varieties of machines for different manufacturing operations. The key features that this factory has: Customer Order Management Production Order Management Material Management Performance Management Customer Relationship Operations Intelligence Quality Management Error-Proofing Maintenance Management Energy Management Workforce Management Environment and Safety We have converted legacy machines into digitized smart machines like lathe machine, shearing machine. The Mobile Collaborative Robot is used for inbound logistics. The following image demonstrates the material and manufacturing process for a typical product like a hydraulic pump flange.  <-Mobile Collaborative Robot Services-> Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Indian Institute of Technology Delhi (IITD) and Automation Industry Association (AIA) joined hands in May, 2017 to set up a center under auspices of Samarth Udyog - a project of the Ministry of Heavy Industries (MHI). The center is called Foundation for Smart Manufacturing (FSM), and it helps supports and develops technologies for right understanding and implementation of concepts of smart manufacturing. FSM is a demo-cum-experience facility in North India, vests in extensive skill building, MSME consultancy, multi-academia partnerships and research which will give a huge fillip to technology aided competitiveness of Indian manufacturing. The project will imbibe technologies from Europe, Japan, USA and India. 'Producing Smarter' with Technologies. FSM has been launched specifically for implementation of smart manufacturing in India which it undertakes to do effectively by: Awareness Building, Prototyping, Simulation and testing Services, Consulting Services, Site Integration Services, Education and Training, Skills Certification, Job work and Research. Mandated Guidelines Awareness campaigns on Industry 4.0 Training for master trainers Active participation provisions for start-up / incubators Hand-holding of SMEs to plan and implement relevant Industry 4.0 projects to be done through consultancy services on a chargeable basis Collaborating with neighborhood universities for student training/internship programs Involving industry in SPV members model for sustainability Participating in a Government-formed platform for Industry 4.0 To make adequate provisions for e-waste management Involving as many clusters of 'capital goods' as possible  At FSM we are committed to demystify the smart manufacturing technologies by learning ourselves and then transfer the learning through various training programs to industry and academicians. We are engaging researchers to indigenously develop technologies to make it affordable by Indian MSME and facilitate adoption of Industry 4.0. Our state of art cyber physical lab showcase technologies available across globe through our strong partnership with the automation industries. Director With a far-sighted approach, the Government of India has supported Automation Industry and IIT Delhi in creation of advanced engineering and software facilities for smart manufacturing. Prospective integrators, consultants and end users can use our training and tryout services to validate their proof-of-concept before scaling up. We help demystify and de-risk your investments in Industry 4.0 Co-Creator Dr. Sunil JhaDirector Mr. Dilip SawhneyDirector  Dr. Anil WaliDirector  Sunil MehtaDirector Mr. Anup WadhwaDirector Automation Industry Association Mr. Ravi AgarwalDirector Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in The digital factory concept offers an integrated approach to enhance the product and production engineering processes and simulation is a key technology within this concept.  Simulation helps to create digital models of products, production systems, logistic systems so that one can explore a system  2019s characteristics and optimize its performance. These digital models have the ability to run experiments and what-if scenarios without disturbing existing production systems or 2013 when used in the planning process 2013 long before the real production systems are installed. Extensive analysis tools, such as bottleneck analysis, statistics and charts give us an opportunity to evaluate different manufacturing scenarios. The results provide us with the information needed to make fast, reliable, smarter decisions in the early stages of production planning. In addition, we can also optimize material flow, resource utilization and logistics for all levels of plant planning from global production facilities, through local plants, to specific lines. This helps to fulfil the user  2019s need to deliver just-in-time (JIT) or just-in-sequence (JIS). Benefits f0a7 Increase existing system productivity f0a7 Optimize resource consumption and re-use f0a7 Optimize systems for reduced energy consumption f0a7 Shorten new product introduction, time-to market, and time-to-volume f0a7 Improve production layout and minimize investments f0a7 Ensure that machines and equipment are in the right place f0a7 Ensure that sufficient material handling equipment is available f0a7 Optimize buffer sizes f0a7 Ensure that product handling is kept to a minimumApart from advance simulation another disruptive innovation is the concept of digital twin. With the growing deployments of Internet of Things (IoT) systems, the importance of the concept of a digital avatar of a physical thing has gathered significant interest in the recent years. A digital twin can be defined, fundamentally, as an evolving digital profile of the historical and current behavior of a physical object or process that helps optimize business performance.A digital twin enables flexibility in manufacturing to reduce time needed for product design, manufacturing process and system planning and production facility design. A digital twin improves quality and even supports new business models that offer opportunities for small- to mid-size companies to expand and bring more high-tech capability into their shops. Digital twins will help all companies become more flexible, reduce time to market, reduce cost, improve quality and increase productivity at all levels of the organization. Digital twin business values: 1. Quality f0b7 Improve overall quality f0b7 Predict and detect quality trend defects sooner f0b7 Control quality escapes and be able to determine when quality issue started 2. Warranty cost and services f0b7 Understand current configuration of equipment in the field to be able to service more efficiently f0b7 Proactively and more accurately determine warranty and claims issues to reduce overall warranty cost and improve customer experiences 3. Operations cost f0b7 Improve product design and engineering change execution f0b7 Improve performance of manufacturing equipment f0b7 Reduce operations and process variability 4. Record retention and serialization f0b7 Create a digital record of serialized parts and raw materials to better manage recalls and warranty claims and meet mandated tracking requirements 5. New product introduction cost and lead time f0b7 Reduce the time to market for a new product f0b7 Reduce overall cost to produce new product 6. Revenue growth opportunities f0b7 Identify products in the field that are ready for upgrade  Improve efficiency and cost to service productThese digital twins use data from sensors that are installed on physical objects to represent their near real time status, working condition or position. Nonetheless, advances in computer science have made it possible to broaden the scope of the digital twin to include many more capabilities, information, inputs and outputs. It also helps you develop and introduce new products to the market much faster than ever. Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in The Mobile Collaborative Robot (MCR) is developed to achieve autonomous material transportation within the Cyber Physical Factory. The MCR consists of a TM5M700 collaborative robot mounted on top of an Addverb Automated Mobile Robot (AMR). The AMR uses a LIDAR and two depth cameras to navigate using Natural Navigation. It has a payload of 150 kg. The AMR runs on Robot Operating System (ROS). The collaborative robot mounted on top of the AMR has a camera on it. The cobot can be programmed using ROS or the TMFlow software. The feed from the camera can be used for object detection. The cobot has a payload of 6 Kg. The MCR can also be controlled using a Fleet Management System. The MCR can be dispatched to various locations in the map using REST Api communication through the Fleet Management System. Education & Training Device Services Coming Soon.. <-Robotic Welding Cell Cyber Physical Factory-> Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Smart Lathe Machine project is about retrofitting of Sensors and PLC for Digitizing the Cutting Parameters. Additionally for the monitoring of Power Consumption, the energy meter is in place to find out the power related parameters. The idea is to digitize the data and add operator and machine safety interlocks to avoid accidents in the shop floor. Also, the digitized data will be used in implementing Industrial IoT dashboards for monitoring purposes. The cutting parameters along with energy data is then used for training the machine model to predict the roughness or the remaining useful life of the tool depending on the vibration signals acquired through the vibration sensor. The Augmented Reality experience is created around the machine for giving basic nomenclature of Lathe machine. The animation of each process is also part of the experience. AutomationIIoTMachine LearningAugmented Reality 01Augmented Reality for guiding the students learn the different components of the machine 04Real-Time Health Monitoring of Assets in Factory 02Augmented Reality with guided exercises for maintenance. 03Machine learning for predicting the tool wear. Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Event DateOngoing Mode Of LearningVirtual IITD-AIA Foundation For Smart Manufacturing presents a comprehensive Learning Module on 201cIndustrial Automation and IoT using soft-PLC  201d Program, visualize, execute remote monitoring, motion programming and more using soft-PLC on the upcoming Learning Module. This program is meant for Engineering Students and Industry Freshers wanting to gain practical learning in the field of Industrial Automation which is foundation for Smart Manufacturing implementation.Learning content prepared under guidance of Prof. Sunil Jha  Event Date16th May -16th July, 2023 Mode Of LearningVirtual IITD-AIA Foundation For Smart Manufacturing is offering an 2 Months Full-time Online summerinternship starting from 16th May 2023 to 16th July 2023 in smart manufacturing domains.Last Date for Application: 15th-April-2023. Event Date20th-24th February, 2023 IITD-AIA Foundation For Smart Manufacturing is delighted to announce the successful conclusion of our week long training program titled \Competency Building in Industry 4.0\ for industry experts from Hindustan Aeronautics Limited (HAL) in the field of smart manufacturing  . HAL has entrusted us with the task of upskilling their experts from various domains. Our carefully crafted learning sessions provided valuable insights into the major domains of Industry 4.0. Including Automation, IIoT, AugmentedReality, Robotics, AI, ML  Event Date15th February, 2023 IITD-AIA Foundation For Smart Manufacturing got an opportunity to be a part of G20 Digital Economy Working Group (DEWG) in Lucknow hosted by UP Government. These type of events helps all the segments of nation like Politicians, Economic dignitaries, Business leader, Entrepreneurs, Industry Associates etc. to come on a common platform and take nation towards a bright and better future and we are proud to be participating and showcasing live demonstrations of our work to the visitors. Event Date22th December, 2022 We are happy to inform that on December 22, 2022, IITD-AIA Foundation For Smart Manufacturing presented various industry 4.0 technologies at Workshop on Automation for Smart Manufacturing organized by Automation Industry Association in Pune. Visitors were given Live Demonstrations of technologies created for smart manufacturing in the cyber-physical lab at the Indian Institute of Technology, Delhi. Here are some of the day  ‘s highlights.  Event Date10th December, 2022 We are happy to report that on December 10, 2022, IITD-AIA Foundation For Smart Manufacturing presented various industry 4.0 technologies at industry day at Indian Institute of Technology, Delhi. Live Demonstrations of Technologies developed in Smart Manufacturing were given to the visitors. Here are some of the day  ‘s highlights. Event Date29th November, 2022 Mode Of Learning Offline (4PM to 6PM) IITD-AIA Foundation For Smart Manufacturing, takes immense pleasure to invite Engineering Students & Fresh Graduates for a Workshop on Industry 4.0 on the occasion of its 5th Foundation Day. Join us for an interactive session with FSM experts on Industry 4.0 & Career Opportunities in Smart Manufacturing domain.Date : 29th November, 2022Timings : 4PM to 6PMVenue : Seminar Room, Room no. 422-B,Mechanical Engineering Department, IIT Delhi.  Mode Of Learning Offline IITD-AIA Foundation For Smart Manufacturing presents Graduate Engineer Trainee programme starting from 1st September.An excellent opportunity for freshers to step into learning smart manufacturing. Benefits:Gain the knowledge about Smart Manufacturing & Automation Technology.Get hands-on experience on a host of cutting edge Industrial Automation equipment.Learn device interfacing, communication, cloud engagement, data dashboarding techniques and much more.Build your career to the next level.Get the opportunity to learn under the guidance of Prof. Sunil Jha! Event Date13th October, 2022 UPSKILLING IS THE NEED OF THE HOURSKILLING OPPORTUNITIES FOR SMART MANUFACTURING REVOLUTIONJOIN US ON OCTOBER 13TH, 11AMRegistration link:https://lnkd.in/dGAk89nwGet to know about New Age Skills that you can acquire and enhance your career success.Join the discussion and get aligned with the future Explore the Smart Manufacturing Revolution with us.  Event Date7th-October We are delighted to share that on 7th Oct 2022, IITD-AIA Foundation For Smart Manufacturing showcased I4.0 Technologies to Honorable Minister (MHI)-Dr. Mahendra Nath Pandey, MoS (HI)-Shri Krishn Pal Gurjar, at National conference conducted by MHI on Industry 4.0 at Kevadia, Gujarat.During the conference, team FSM demonstrated Self Guided Maintenance on the Smart Training Kit using Augmented Reality, Remote Monitoring and Control of the CP Lab at Indian Institute of Technology, Delhi via IIOT where the visitors were able to place order and monitor the progress on the Live Dashboard. Event Date30th July, 2022 IITD-AIA Foundation For Smart Manufacturing along with Indian Institute of Technology, Delhi participated in the \Academic Outreach Program\ held at the IIT-Delhi Sonipat campus. All the participating academic and skilling institutions in this program expressed their appreciation for showcasing the latest smart manufacturing and automation technologies. FSM initiatives for creating a platform for exploring and implementing smart manufacturing concepts and practices in India were very well received by students, academic and industrial institutions.The event focused on the sharing of knowledge for solving problems in manufacturing through automation and smart manufacturing technologies thereby helping to increase the competitiveness of the domestic manufacturing sector.  Event Date1st June-31st July, 2022 Mode Of LearningVirtual / Physical IITD-AIA Foundation For Smart Manufacturing is offering 100 openings for 2-Months Full-time Online Summer Internship starting from 1st June 2022 to 31st July 2022 under the theme of 201cSmart Factory  201d. During the months of June-July 2021, FSM organized a Summer Online Internship. 900+ applications were received and 150+ students were accepted for internships in various domains such as Augmented Reality, Machine Learning, Automation, IIoT, Realtime Dashboards, and Robotics. More than 45 projects were conceptualized and completed during the internship and 20+ Research Papers were prepared by the interns. The FSM Masterclass Series was a 4-month program, having 4 different masterclasses for technologies such as 201cAugmented Reality  201d, 201cMachine Learning  201d, 201cAutomation  201d, and 201cIIot  201d. The program was attended by college students, college professors and employees from various industries. Each course was a month-long program, consisting of 4 live classes, 4 hands-on lab sessions, e-learning modules and 1 major project.  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Application DevelopmentWeb DevelopmentMobile Application DevelopmentFirebaseProgramming LanguageC++FlutterReact JS  Hi, I am Sanjeev Kumar. I belong to Meerut and I have done my Bachelor  2019s Degree in Information Technology (B.Tech.) at JP Institute of Engineering and Technology, Batch 2016-2020. After completing my graduation, I did my internship in a project-based company in Noida. I like coding and logic-building programming. I developed many mobile applications just for improving my skills like Student Management System, Location-based reminder, and incoming calling auto recording application. My next initiative would be to develop a video calling and chatting application. In the area of my interest, there are lots of things that i enjoy like traveling, video games, reading books, movies, and music. I joined FSM on 7th Feb 2022 and I like the environment of this company. I learned many things in FSM about automation and IIoT. Their are very supportive people in FSM and treat everyone like a family. Every time when we face any challenge, regardless of the individual  2019s domain each team member steps up to the challenge together. I enjoyed working with this team during many events like DD Robocon and ABU Robocon. I would like to thank Arun sir for his constant support and guidance. He has been like a pillar for me in FSM. I started my journey in IAFSM on 7th Feb 2022. This platform is amazing. Before working here, I was in Surat, Gujarat in an E-Commerce Company as a Software Developer. I am good at Mobile Application Development. I work on the Web as well. In IAFSM, I work under the leadership of Mr. Vaibhav Anand. He is good as a mentor. We did many projects together. I was part of DD Robocon 2022 & ABU Robocon 2022 events as an organizer team. I work on Flutter, Node JS, Python, PHP, WebRTC, Joomla, React JS, and Firebase.  Projects IAFSM Lab Autonomy Workstations Order Application MPRC Dashboard MPRC Order Application CP Factory Application Workstations Dashboard DD Robocon User and Admin Website ABU Robocon User and Admin Website Smart Warehouse IAFSM Website Management Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Multi Process Robotic Cell Smart Lathe Machine Robotic Welding Cell FSM AR Demo Mechanism Kit Assembly and Disassembly AR Maintenance Using HoloLens Expert Maintenance System Using HoloLens Cyber Physical Assembly Line CP Lab Autonomy Smart Mechanism Kit Realtime Values PLC Kit Parts Introduction  Smart Mechanism Kit Motor Replacement Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in  The Automation Industries Association (AIA) in its quest to set up a Common Engineering Facility Center (CEFC) at IIT Delhi has brought together experts from various industries in the sector of automation. These experts look forward to work together and use their industrial experience along various verticals to initiate smart manufacturing in India. The following firms have agreed to come together to set up the CEFC. They comprise Technology & Investment Partners, Simulation &amp; Integration Partners and Machinery Partners.  We envision integrated technology advancement in the Indian Manufacturing Sector to unleash the hidden potential of small and medium enterprises. We aim to achieve this by carrying out a holistic intervention called Smart Manufacturing; that imbibes innovative approaches around which smart technology enabled manufacturing (STEM)can fructify. Many aspects of Industry 4.0, Industrial Internet Consortium and e-Factory can be broken down into 201cautomated practices  201d that can incrementally be connected into a manageable and logical process to suit short-term needs of investors. This practical and cost-effective approach would also generate a natural momentum toward long-term goals. The national Common Engineering Facility Center will act as the medium to accomplish this vision of developed India by integrating the research expertise of IITD and invaluable practical experience of industry. On assessing the exciting advantages as well as various issues we face in adopting Industry 4.0, there is one overriding conclusion that we all need to acknowledge the dream of wanting to somehow just jump in and excitedly resolve the entire implementation challenge all at once.  For one thing, requirements for Industry 4.0 are just too comprehensive and unwieldy to consider tackling it all at once. For another, Industry 4.0 will likely take years to be commonplace. But in nutshell, Industry 4.0 will never be 201ccomplete  201d anyway, it is a continuous-improvement tool, flexibly changing and expanding its iterations as the world evolves. The Smart Manufacturing initiative will revolutionize the production operations in SMEs by improving their quality, reducing throughput time and inventory cost. New analytics solutions help our Indian customers move ahead on their Smart Manufacturing journey, no matter where they are today. This will lead to a simplified approach to analytics converting data into decisions. This will also enable flexibility to the manufacturers to customize their products to cater the high expectations of their consumers. This will also improve the quality of jobs and uplift the standard of living of people.  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Being able to assist customers remotely, service technicians could support their customers at any time and from anywhere, without necessarily having to visit the customer site. Remote access could also help to better prepare service technicians with the necessary information for their tasks. Knowing about the machine or system error in more detail beforehand saves valuable time because required spare parts or other equipment can ordered, prepared and brought along. Moreover, due to a shorter reaction time, the customer  2019s machines are up and running much faster, saving them from costly downtime.  For a machine and equipment manufacturer, using remote access also means that the same number of service technicians could support more customers or offer additional services.  A secure remote connection to distributed machinery and equipment is also the basis for many new concepts and services within Industry 4.0 such as predictive maintenance, where a secure connection is established to collect data from machines, equipment or devices. The data is than analyzed and used to detect errors and possible failures at an early stage to avoid unplanned downtime. Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in 3D printing has the capability to be used to respond too many of the world  2019s changing megatrends. The physical object is made from a three-dimensional digital model, typically by laying down many thin layers of a material in succession. With 3D printing it is possible to go directly from digital design data to a final part with no intermediate production steps. 3D printing technologies therefore eliminate the need for tooling and the associated capital investment. The result is that companies that adopt 3D printing can disrupt the traditional economies of scale, by allowing cost-effective production of single-unit or low-volume batches with low-volume part production, products can be customized to local markets, or even to individual customer tastes, driving adoption within industries as diverse as fashion, health care and automotive. Moreover, with the ability to print on demand, businesses also have the opportunity to eliminate inventory and cut aftermarket lead-times by providing digital spare-parts catalogues that can be printed when needed. The smart companies of the future will be those that have a clear strategy for how and where 3D printing fits within their supply chain and their value chain. They will also understand the business drivers to adoption.  From the student  2019s study to the professional designer  2019s office, from the dental laboratory to the jewellery retailer, from the aerospace factory to the hospital basement, 3D printers have become invaluable business tools. Applications and reasons are as diverse as the users. What links all these applications and users is one underlying ability: to transition 3D information digitally and seamlessly from the virtual world to the real world with nothing but a computer and a3D printer 2013 from bytes to bits.  The benefits of 3D printing today go way beyond just making models and prototypes. 3D printing is becoming a way of making components, systems and products sold across the supply chain, from the bracket in the aircraft door to the dental aligner in the teenager  2019s mouth. It can also be used for the manufacturing of tools, jigs and fixtures in addition to rapid prototyping. Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Short term education training offered along with online learning and assessment tool. Participants are engaged for approximately 100 hours in different technologies that include Automation, Industrial IoT, Augmented Reality, Robotics, Digital Twin and etc. The learnings are divided into Awareness Level, Learning Level and Expert Level. Smart Manufacturing Training Kits Software CPS facilities and the associated technologies will provide for one-stop shop for users whose needs are cut across multiple disciplines and who wants to witness comprehensive digital transformation before taking up the actual implementation. Cyber Physical Assembly Line Multi Process Robotic Cell Robotic Welding Cell Mobile Collaborative Robot  Cyber Physical Factory We are providing following services: 3D printing Smart Manufacturing Training Kits for Industrial Automation learning MIG Welding (including Wire-arc-additive-manufacturing)  Multimaterial 3D printer Smart Manufacturing Training Kits MIG Welding Cell Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in RoboticsPlanningCAD  Retrospecting, it has been almost 3 years being a part of the FSM family. Being an automobile graduate with experience of Supra SAE and working short term in Automobile service sector I set foot in this fraternity. It filled me with immense happiness and a great sense of responsibility for this was not a matter of just work and salary but was much more than that. Look at the objectives of IAFSM and you would feel the same. Initially familiarized myself with the smallest electrical component such as lugs/ thimble to fabrication of mechanical components of the workstation that were in the process to be erected, I was just feeling the air of Industrie 4.0 . We had almost every components lying on the FSM lab floor, waiting to be assembled and programmed. That was the moment to say no to nothing. Every step was a learning. To my unfathomable surprise, soon I was assigned project Multi process robotic cell in 2018. It was my first step in the field of robots. So there were technical areas being learnt and explored like the utility of a robot in a tight yet safe environment with sensors, pneumatic, but visualization, diligence and team work was of quintessence. It made everything possible with the guidance of our Directors. It was a comprehensive learning with blend of things involved like ideation, brainstorming, documentation, versioning, robot programming, gist of PLC programming, software utility, sensor implementation, data flow to Cyber space, mechanical hardware, human interaction with different people involved with various personalities, given all this the project had to be on track. The attempt to do so was one of the biggest learning. So I remember, that I may not do everything correct but be humble enough to accept my failure, rectify and move forward. That helped to raise confidence and work enthusiastically with my team mates. I take them seriously and respect them. I remember the event we had gone as a team of two and represented our fraternity at IMTEX Bengaluru as ambassadors of the Industry 4.0 revolution and an opportunity filled with pride and confidence. Exposing ourselves to a pool of machinist, industrialist and young enthusiast was breaking shackles of comfort and dawning fresh energy upon us. Another opportunity of Robotic Welding Cell and something that is quite challenging. Initially, I had a new team of students from an engineering college to ideate and work with. Mentoring and visualizing at the same time was not always easy. I had to update myself with novel approaches and keep on trying different methodologies whatever involved in the project. It was interaction with the students, brainstorming, concept building of the project, utility of the technology especially the welding process with two robots, documentation for clarity of purpose, implementation and more. The high level of mentoring, guidance on the technical, managerial, spiritual areas always provided a boost and relief at the same time by our Directors, our dignitaries, our gurus. Their level of energy resonated within us and is the reason what I am today. I am, will be always grateful for this experience that am sharing. Thanking my team mates for their continuous support they have granted me. I am keen on learning various disciplines that involve the field of robots and robotics and prosper in the same direction. Projects Executed: Multi Process Robotic Cell Robotic Welding Cell  Developed a robot welding cell in a compact ensuring safe environment safety equipment's and with sensors, electro-pneumatic and master controller (PLC). The robot for welding, material handling and cell development are as per ISO standard 10218-2. Prepared the Functional Descriptive Specification(FDS) document, covering project objectives, tentative layouts, system components, control architecture and important milestones for Robotic Welding Cell project Evaluated multiple Material Handling Robots, Welding Robots and MIG Welding Power Source for tendering for Robotic Welding cell project from companies like KUKA, ABB, Yaskawa and Omron Techman. Developed robot welding simulations to study the reachability and collision-free environment for both robots. Generated offline robot program (OLP) and calculating cycle time. Software used are KUKA Sim4 and RoboDk. Played an lead role in Ideation & Concept Development of Robotic Welding Cell project and in collaboration with my team for Multi Process Robotic Cell Analyzed functional specification requirements, finalized BOM, prepared & released tenders for parts procurement and fabrication for Robotic Welding Cell project and Multi Process Robotic Cell project Ideated and designed electro-pneumatic jigs and fixtures for sub-stations performing various assembly operations of the job in the Multi Process Robotic Cell. Designed Siemens HMI following creative Siemens HMI MasterClass. Closely coordinated, escalated and supervised multiple vendors and fabricators to ensure timely completion of project activities and fast tracking the ordered items. Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in IITD-AIA Foundation For Smart Manufacturing presents a comprehensive Learning Module on 201cIndustrial Automation and IoT using soft-PLC  201d Program, visualize, execute remote monitoring, motion programming and more using soft-PLC on the upcoming Learning Module. This program is meant for Engineering Students and Industry Freshers wanting to gain practical learning in the field of Industrial Automation which is foundation for Smart Manufacturing implementation.Learning content prepared under guidance of Prof. Sunil Jha  IITD-AIA Foundation For Smart Manufacturing presents a 5 Days/10 Hours Online Self Learning Program on components and integration for Smart Manufacturing. This program is meant for Engineering Students and Industry Freshers wanting to gain practical learning in the field of Industrial Automation which is foundation for Smart Manufacturing implementation.Learning content prepared under guidance of Prof. Sunil Jha IITD-AIA Foundation For Smart Manufacturing presents a 5 Days/10 Hours Online Self Learning Program on components and integration for Smart Manufacturing. This program is meant for Engineering Students and Industry Freshers wanting to gain practical learning in the field of Industrial Automation which is foundation for Smart Manufacturing implementation.Learning content prepared under guidance of Prof. Sunil Jha  IITD-AIA Foundation For Smart Manufacturing presents 4 hours free awareness course on Industrial Automation Learning. This is a supplement course helpful for the upcoming 5 Days/10 Hours Online Self Learning Program on components and integration for Smart Manufacturing Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in  Short-term education and training courses will be offered along with on-line learning and assessment tools to educators and users on a chargeable basis. This new form of collaboration opens up previously inconceivable possibilities for the smart factory of the future. As collaborative robots operate without physical safeguards, they have to permanently calculate the risk of colliding with humans, constantly checking this via the robot controller.  Collaborative robots works hand in hand with the operator, thereby enabling him to work more efficiently, more ergonomically, more precisely and with greater concentration. As a robot that can genuinely be deployed universally, it is defining new standards on the road to the fourth industrial revolution. Collaborative robots can be programmed onsite by the employees and can be quickly re-tasked to do multiple jobs quickly. They are lightweight and can be moved around easily in the production facility.  Collaborative robot have integrated sensors, passive compliance or over current detection as safety features. The integrated sensors will feel external forces and, if this force is too high, the robot will stop its movement. Passive compliance is produced by mechanical components. If an external force acts on a joint, this joint will submit itself to this force. So, in case of a collision, the joint will move in the opposite direction avoiding any injury. Also, an over current can be detected when a collision occurs. This is another safety feature, because the software can generate a security stop when it detects a current spike. Some of the applications of collaborative robots can be:  Assembly Soldering Vision based quality inspection Machine Tending Gluing Screwing Pick and place Operating hand tools Laboratory work  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in AutomationIIoTRoboticsMarketingExecutionPlanningIndustrial AutomationPythonPLC ProgrammingC#HMI DesigningNetworking and Communication  Hi, I am Arun and I am working as a Senior engineer in FSM. I did my B.tech in Electrical and Electronics Engineering. I have been working under the supervision of Sunil Jha sir for last 6+ years. I was working as a Project Assistant (2016-18) in IIT under Jha sir, where I got to learn some automation technologies like pneumatics, electro pneumatics, PLC programming, HMI designing etc.. All these technologies were just a word for me before I started working on them. It was a time where we had resources and exposure to play with these technologies. In 2018, I joined FSM and there I learnt how to implement the knowledge, which I had gained so far. It was a new experience but a very exciting one. Before FSM, I used to get some training kits or individual components to play with but in FSM, I got a whole assembly system to test my knowledge and exceed my own expectations. The freedom, which FSM provided me to work, was something that helped me to flourish more in my field. After completing the programming of assembly stations, I got opportunities to learn other technologies. So, this is the place where I kept getting learning new things while working on a specific project. I have executed multiple projects like Cyber Physical Assembly line, Smart trainer kits, MPRC, CP lab autonomy and also designed training courses on Automation and IIOT. I have been working on Robotic welding cell programming for last 2-3 months and working on this cell gives me satisfaction of completion every day as I get to know something new. So, after working 4+ years in FSM, I have gained knowledge in different domains like Automation, IIOT and Robotics. FSM has a wonderful team to work with. I remember there were times when we had to complete some project before the deadline (if there is any event or program) and the way this team responded was exceptional. We were staying late and stayed some nights also but it didn  2019t affect any of us. Working with this team was interesting and we were enjoying working with each other. There was a training program for 5 days in Sonepat. If someone had looked from outside (who was not part of the team), he must have thought that this was a very hectic schedule as we were working continuously in those 5 days. However, for the team it was like a picnic as we enjoyed ourselves working and helping the trainees. We represented FSM in some of the big events like Imtex Bengaluru, IIOT events, Training programs in Sonepat and FSM etc. Today also when we get a new project or we face any challenge, whole team stands up together to complete the project in the given time. One of the biggest achievements in FSM for me is winning the Best of Nation from India in BRICS Future Skills Challenge 2020 in collaboration with my colleague Ms Preeti Joshi. We worked on Thingworx to design Dashboards that can remotely monitor and control the robots present in Russia!!! I have been working in FSM for the many years but I still feel the same enthusiasm and excitement when I start doing something new or different. This team has a great mentor and guide in the form of Prof. Sunil Jha sir as he gives freedom and guidance to complete any project. Projects Executed in FSM (Senior Engineer) (Joined in October 2018) : Cyber Physical Assembly line Smart Trainer Kits Multi Process Robotic Cell Robotic Welding Cell CP Lab Autonomy Training Courses on Automation and IIOT- Automation and IIOT on Codesys Training manual on Famic Automation Studio Training manual on Rockwell, Siemens and Mitsubishi smart trainer kits. Data Acquisition and Use of IOT Gateways Projects Executed in IIT Delhi (Project Assistant) (August 2016 - September 2018): Programming and Testing of PLC Training Kits Commissioning and Programming of 3 Axis parker system Ball end magneto rheological finishing machine (BEMRF) - Wiring Diagram, CNC programming to polish different types of material Commissioning of Advanced Micro Milling Machine (AMMS) - Panel Wiring, wiring diagram, Integration of parts Created Ladder Logic solutions for all the libraries in Famic automation studio for B.Tech and M.Tech labs Created lab manual on Famic Automation studio Commissioning of 3 Training Kits - Elevator system, Traffic light system and Cooling system Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Augmented reality is one of the cutting edge technologies involved in the industry 4.0 trend when talking about smart manufacturing, it is a technology which was seen just as a fancy toy until a few years back, but which has now reached the right level of maturity to be employed in a productive environment. AR applications can become even more powerful by augmenting them with data generated by another set of technologies, revolutionizing the marketplace called the Internet of Things (IoT). Combining AR and the IoT allows employees, such as field maintenance workers, to access vital machine operating statistics to assist in maintenance decisions, or an assembly worker to visually receive pick and place instructions and alerts if tools are working incorrectly or are out of tolerance.  Key AR ecosystem components include device hardware, an OS open to third-party application development, and a platform for software, as well as content creation and management. The standard device hardware for viewing AR applications has predominantly been smartphones and tablets due to their wide availability and exceptional compute power. The newer devices expected to accelerate enterprise AR adoption because they allow hands-free access to information are smart glasses, HMDs, and other digital eyewear solutions from companies such as Microsoft, ODG, Epson, and Vuzix.  Augmented reality renders 3D objects in real time into your field of vision. For example, looking through the cover of a machine and seeing its internal component structure.  Augmented Reality (AR) is a technology enriching the real world with digital information and media, such as 3D models and videos, overlaying in real-time the camera view of your smartphone, tablet, PC or connected glasses.  On the spectrum between virtual reality, which creates immersive, computer-generated environments, and the real world, augmented reality is closer to the real world. Augmented reality adds graphics, sounds, haptic feedback and smell to the natural world as it exists. Both video games and cell phones are driving the development of augmented reality.  The methods by which AR is achieved can generally be split into two broad categories: Marker-based, and markerless.  Marker-based Systems: This technology uses physical-world symbols as a reference point for computer graphics to be overlaid. For example, a 2-dimensional printed marker is placed in front of a webcam. The computer then interprets this symbol to overlay an on-screen graphic as if it were directly on top of the marker in the physical world. There have been several notable uses, most commonly in marketing  Markerless Systems: By contrast, this technological approach has given rise to 2018mobile augmented reality  2019, denoting use of the technology with devices such as smartphones and tablets. This method uses a combination of an electronic devices  2019 accelerometer, compass and location data (such as the Global Positioning System 2013 GPS) to determine the position in the physical world, which way it is pointing and on which axis the device is operating. This location data can them be compared to a database to determine what the device is looking at, and thus allows computer data/graphics to be displayed on-screen.  Applications of AR in industrial environment: 1. Data Reuse: AR applications allow more efficient reuse of existing enterprise application data. Enterprise application data can include maintenance records, service tickets, and usage logs  2014any combination of which can be tapped into to decorate an AR experience that blends the physical and digital worlds. 2. Data Contextualization: AR applications can be used to better contextualize information across a range of industries and job functions. In manufacturing, this could mean giving linemen and plant personnel the ability to abstract backend systems information for 3D product navigation, step-by-step work instructions, and remote visual guidance. 3. Work Error Reduction: AR applications can reduce errors made by workers in various occupational tasks. Whether the error is caused by too little information, too much information, or poor quality information, AR applications place the right information and instructions at the location of the work task. 4. Workforce Multiplier: AR applications are a workforce multiplier in two ways. First, training resources are more effective and efficient. Training hours can be reduced because AR applications can include video instructions on repair/replacement tasks overlaid on the machines themselves. Second, workers can do more by themselves versus sending multiple workers or truck rolls to address the same problem or work task. 5. Safety: AR applications can also improve the safety of a work task. For instance, AR applications can include not only written instructions for removing or replacing a part on a machine, but can also include visual instructions. This assistance avoids unnecessary and possibly unsafe interaction with the machine or object.  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in An AR application to show information about FSM Smart Manufacturing Kits An AR application that gives the user a menu with 10 options for each of the FSM Smart Manufacturing Kit. The user can visualize all 3 components, the HMI, the Electronic Kit, and the Mechanism Kit. The user can click on the different parts and get to know about the parts. The user can also initiate the CAD explode view animation.  Cloud DatabaseAugmented Reality 01Self-Guided Training Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in An Augmented reality application that allows the user to understand about the mechanism kit and scan the image target and place a CAD model of the Mechanism Kit to understand it clearly. The user can create a new account and can access the application using the registered email id and password. If the user forgets the password associated with the registered email id, they can use the forgot password option and can create a new password for the registered email id. The user can the access the features of the application by using the main menu FSM Mechanism Kit Assembly and Disassembly 2 where different buttons are linked with the assembling and disassembling. The user can then play the animations of the screws, Allen Keys, screwdrivers, and the parts of mechanism kit. The steps can be played multiple times as per the requirements. The user can switch between the steps using the next and previous buttons. Cloud DatabaseAugmented Reality 01Self-Guided Training 04Self-Guided Maintenance 02Action Latency Reduction 03Remote Maintenance Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in. An AR application designed to run on HoloLens 2 to demonstrate the entire sequence for Motor Replacement in the FSM Smart Mechanism Kit.The application uses a model target to detect the real mechanism kit and then superimpose the AR kit.The application will guide in tool selection and then show the steps to replace the motor.Voice updates are given to the user with each step.The user can also control the application sequence using voice commands. Augmented RealityMicrosoft HoloLens 2 01Self-Guided Training 04Self-Guided Maintenance 02Action Latency Reduction 03Insight Latency Reduction Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Workstation 1 is one of the assembly stations. This Station stores the raw part and inspects the parts using QR code scanner and vision sensor. Valve body is transferred from one station to another in a pallet. Pallet has a RFID tag,so that next station can read the value and get the information about the body present in the pallet. PLC Used: 1769-L33-ERM,Micrologix 1400 (Rockwell) Software Used: Studio 5000,RSlogix 500 and View ME(HMI designing)  Programming: Ladder Logic Programming, Safety Sensors (IO Link master), Pneumatics and 4 Axes Motion Programming. Workstation 2 is the next assembly station. This station inserts the child part in the base part. PLC Used: BnR Software Used: Automation Studio Programming: Ladder Logic/Structured text Programming, Safety Sensors (IO Link master), Pneumatics and 3 Axes Motion Programming. Workstation 3 is one of the assembly stations. This Station assembles all the sub parts of Cotter joint assembly depending on the type of socket body approaching the station with the help of RFID. The Gripper Picks and Places the Cotter to engage socket and spigot. PLC Used: Mitsubishi MELSEC R-Series (Rack Type) Software Used: GX Works 3  Programming: Ladder Logic Programming, Safety Sensors, Pneumatics and 7 Axes Motion Programming.  AutomationAugmented RealityIIoTMachine LearningDigital Twin 01Demonstration & Training 04Flexible Assembling 02Customer order processing 03Realtime dashboard Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Software used for AR : Here are various software we are using for the smart manufacturing purpose in AR. It is an AR application designed and developed to provide a complete 3D CAD model of the FSM Smart Kits. The application uses a pre-defined Image Target and superimposes a 3D CAD of the Smart Manufacturing Training Kits. The user can visualize the smart kit without having access to the physical kit. He/She can click on the individual parts to display information about them. He/She can also get an exploded view of the CAD to understand the disassembly process. All activity of the user is also being logged on the cloud and displayed on a dashboard, this allows us to track the progress of the user. It is an AR application which uses the physical FSM Smart Kit as an Image Target and adds information about their individual components. The app also allows the user to click on the individual parts and learn about them. It is an AR application to help the surgeon in verifying and sensing the correctness of the bent plate. The application uses a 2D Image Target to superimpose the CAD of the bent plate on the Image Target. The surgeon will try to match the physical plate to the superimposed cad to confirm the correct bent. It is an AR application to guide the surgeon for drilling correct holes in the spine to insert screws. The application uses a 3D spine as the Model Target, then superimposes the screws on the CAD. This will guide the surgeon to determine the angle and location of the screws. It is an AR application that provides a step-by-step process to assemble and disassemble the FSM Mechanism Kit. It is an AR application that provides the user a 3D visualization of the Cobot joints' movements. Use sliders to move the joints and learn about the working of Cobots. It is an AR application to scan the surroundings and place full-scale objects to confirm size requirements. It is an AR application to get a 3D view of all the smart machines and kits at FSM. It is an AR application that replicates the assembly process of the assembly line. It gives the user a complete visualization of every step of the assembly process. Smart Catalogue & visualization Education & Training Use as a teaching tool Assess machinery before purchase Maintenance Tool <-Smart Manufacturing Training Kits Cyber Physical Facilities-> Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in  IITD-AIA Foundation for Smart Manufacturing433B, Block-2, Department of Mechanical Engg.IIT Delhi, Hauz KhasNew Delhi - 110016 Call us for any query011-26582053, 8076197190 info@iafsm.in The term safety denotes the functional safety of machinery or, put another way, the protection of people and the environment against threats that can proceed from machinery. Safety demands that residual risks arising from a plant or machine do not exceed acceptable values. This includes hazards to the plant environment (e.g. environmental damage) as well as hazards within the plant or machine (e.g. people inside the plant). One option for the worst case is simply to interrupt the energy supply straight away and bring the machine to a hard stop. The traditional way of providing scope for this is by means of special safety wiring and components such as safety relays. Because this approach is very much hardware-based and therefore static, it is not particularly suitable for intelligent manufacturing processes where plant layouts continually need to be changed. A hard shutdown is generally associated with further disadvantages, whether these involve loss of productivity, extended downtimes due to more complex recommissioning procedures or a restriction in the machine  2019s operating and maintenance concept.  An alternative is offered by dynamic safety concepts based on an integrated view of changing automation processes and functional safety requirements. This changes the view of safety itself; it is regarded less as a hardware characteristic and more as a cross-device function. This approach allows processes to be operated in a safely controlled manner without any need to interrupt them immediately every time a fault occurs. But the dynamic approach can only be implemented efficiently if functional safety is built into automation projects from the moment they are planned. Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in A system consisting of 3 parts 1. Python code 2. Notification System 3. AR Application The system is designed to automatically detect any error occurring in the Work Station 1 and raising an alert, the user then using HoloLens 2 will follow the steps to resolve the error IIoTAugmented Reality 01Self-Guided Training 04Self-Guided Maintenance 02Action Latency Reduction 03Insight Latency Reduction Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Cyber Physical Assembly Line demonstrates smart manufacturing and comes with different functionalities. It includes the Custom order processing, valve body, spool, solenoid, connector, valve functioning testing, packaging, inkjet printing. It consists of three work stations and one MPRC.Workstation 1: It is also known as Raw part storage, part loading and quality check. It uses the Rockwell PLC. It stores Product and do Pick & place arrangement for mounting the product on pallet.  Workstation 2: It is also known as axisymmetric part storage quality check and inspection work station. It uses B & R PLC. It stores spool and do pick and place arrangement for inserting the spool in valve body. It uses proximity sensor for inventory check. Workstation 3: It is also known as Prismatic Part Assembly and Screwing Workstation. It uses the Mitsubishi PLC. It stores all prismatic parts. It is used to place and tighten all the parts to valve body.Multi Process Robotic Cell (MPRC) :Click here to know more. Conclusion: All the station can run in Auto/Manual Mode. This is how the cyber physical assembly line works. There are features in the line such as smart sensing, Machine to Machine communication, smart dashboard, remote monitoring, alerts and notifications on email, safety, remote maintenance and Augmented Reality.  Simulation & Testing Services Education & Training Prototyping, Consulting Services Skills Certification Site Integration Services Job Work & Research Multi Process Robotic Cell-> Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in An Augmented Reality application to show the Real-time Values of the 4 stations of the FSM Smart Mechanism Kit by connecting with KEPServer and getting tag values Augmented RealityIIoT 01Self-Guided Training 02Realtime Monitoring Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in The \Competency Building in Industry 4.0\ course covered essential topics for implementing Industry 4.0 technologies in the industrial sector. Participants learnt about IIOT connectivity, creating digital SOPs, part tracking, and how to simulate a robotic cell. The course also covered the use of cobots for pick and place operations in assembly lines. These carefully crafted learning sessions provided valuable insights to the participants into these major domains, and they will be able to apply these learnings in their unique use cases. By the end of the course, participants got the necessary knowledge and skills to stay competitive in the evolving industrial landscape. Implementing IIOT Connectivity Implementing IIOT connectivity on an Assembly Line.  Mapping Data and creating Dashboards Finalizing Data and Monitor Realtime data on an IIOT platform to monitor Data of the Assembly Line Creating IIOT Dashboard of the Assembly Line Designing a front-end UI for the operator to operate the parameters/tags of the program in the HMI.  Creating Digital SOPs + Part Tracking Creating Digital SOPs for the Assembly line Tracking a part in an assembly line (RFID)  Robotic Simulation of a Robotic Cell Pick and Place of a pallet in an assembly line using cobot  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Robotic Welding Cell demonstrates a flexible manufacturing cell meaning it has an inventory of its own, material handling system, variety of parts with dynamic-order.This welding cell can be a part of production line and act as standalone system.Use of single or multiple robots may involve use of a PLC also.This cell uses a saftey PLC.This cell has all these features and the basis of selection can be done in a way that the cell will be programmed, utilization of peripheral devices (proximity sensors, pneumatic cylinders), safety devices and Automatic Identification & Data Capturing (AIDC) devices like RFID, vision camera etc. The cell has following level of flexibility but the decision can be made on the basis of the flexibility required as per the following parameters:1. Variety of parts for simultaneous production 2. Change of schedule3. Ready for new partWorkstations and layout design :Workstations in the cell are as per the following given applications and subject to following parameters : 1. Load/Unload stations for the cell: It is the area where the raw materials are incepted in/out of the cell.2. Inventory storage area: An inventory storage where the storage of the raw material is kept.3. Inspection area: An area where the material post operations are inspected for quality. This is kept as a separate unit away from the cell but it can be an integral part of the cell itself.4. Operations area: An area where the operations like welding is performed. In the cell , this area is more of a workspace since two robots are responsible for performing welding on the material. And it can be anywhere in the robot's workspace. 5. Layout design: It is an integral part before identifying the robots and environment preparation and also plays a key role in safety of the cell.CAD Design & Simulation: The 3D visualization and detailing of the cell is done using a designing software. The robotic path simulation software asists in defining the best possible, collision free environment that is going to be the basis of the layout design and all other hardware that are going to be installed in the cell. Apart from designing & fabrication of the cell it helps in performing the robot simulations to generate offline program, collision free environment (this is how the level of flexibility achieved) for robots at operating stage and generating Augmented Reality applications.  The cell programming can be done in atleast two ways. Online programming where teach pendant is used and other is offline programming where the simulation is done in a separate software like RoboDK and KUKA Sim4. This helps in visualising and performing collision free path generation for both the robots. The video below demonstrates the offline programming method where we can see two robots performing indexed type motion. The offline programming helps in virtual commissioning and without impacting the production. Simulation & Testing Job work & research Prototyping Customer Order Proessing Site Integration Machine-as-a-service <-Multi Process Robotic Cell Mobile Collaborative Robot-> Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Smart trainer kits are used to train industry and academic people and make them aware about Industry 4.0 technologies. It is a combination of three independent hardware accessories- PLC Trainer Kit, HMI Kit and Mechanism Kit. All these kits demonstrate the industry 4.0 technologies in the best possible way. These were designed, manufactured and integrated in FSM. Modules/Trainings covered in these kits: PLC Programming, Network Protocols HMI Designing Data Logging Dashboard Designing, NodeRed IIOT gateways and platforms Augmented Reality Machine Learning Smart Sensing Motion configuration and programming Vision System  This is one of the kits of the smart manufacturing trainer module. This kit has the brain of the other two kits (HMI Kit and Mechanism Kit) as programming is done in this kit. It has a PLC for programming and digital inputs/outputs, analog inputs/outputs to connect the required sensors with the PLC. It has an IO link module to connect the smart sensors, so alongwith the 24VDC or analog input this module communicates the sensor status with the PLC. Data logging and data collection is one of the critical steps for IIOT and for that we have integrated IOT gateway and other software platforms in this kit. To connect sensors with the PLC, banana connectors are used. It can communicate on a number of network protocols and it has the feature to communicate with multiple HMIs and Servo drives. In programming kit we have PLC hardwares from Rockwell, Siemens, Schneider, Omron, Delta, Turck and Mitsubishi for programming and various softwares. NUC is also integrated in the kit so that it can be used as CPU for softwares and PLC programming. So, this kit is more than sufficient to demonstrate the features of the PLC, gateways and software platforms (Data Logging, Augmented Reality ,IIOT gateways and platforms), Smart Sensing . Human Machine Interface is used to monitor and access the PLC data. Screens can be created depending on the application.This is the second kit of the smart manufacturing trainer module. It has HMI to display the screens and monitor the PLC tags. The hardware buttons and LEDs as well which are connected with the PLC trainer kit using banana connectors, mostly like plug and play. These are used to test the PLC program and as input and output in the PLC program. HMIs used in these kits are smart HMIs which have smart features like OPC UA server/client, MQTT compatibility, user authentication on the screens, act as an edge device etc. So, kit is used to visualise the PLC logic. Additionally it demonstrates the HMI Designing and Dashboard designing feature. This is the third kit of the smart trainer modules used to demonstrate a working manufacturing system. It has a servo motor and drive to rotate an indexing table. Along with the motion feature, it has four stations which can be used to demonstrate multiple operations. Each station has smart inductive sensors to sense the part's presence which consists of the distance and proximity sensors, servo drive, test product, RFID Reader, Solenoid, USB Camera mounted on them.Station 1: It has a RFID controller used to read and write the tag values of a product. It communicates with IO Link master (present in PLC trainer kit) and IO link master communicates the data with the PLC. So, RFID operation and use case is demonstrated. Station 2: This station consists of a distance sensor used to calculate the height or distance of the product. It is a smart sensor and communicates with IO Link master. So, along with the analog value it communicates it  2019s status to the PLC as well. Verification of product dimension is also demonstrated. Station 3: This station has a vision camera installed on it. This camera is used for image processing and product testing. Sorting operation is performed using the image processing feature. Station 4: This station has a solenoid which is directly connected with the digital output of the PLC. Extension and retraction of the solenoid is programmed in the PLC for this operation.This mechanism kit is used to demonstrate and train motion configuration and programming, Vision System, Smart Sensing, Machine Learning, Augmented Reality.  Programming Kit HMI Kit Mechanism Kit It is showing the working of Smart Manufacturing Kit Education & Training Device Services Research & Skill Certification  Software-> Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Multi Process Robotic Cell demonstrates a flexible manufacturing cell. This cell is an assembly line where components of a main part gets assembled, the main part comprises of minimum of 7 sub-parts and more, which are then assembled by placing, inserting and part-screwing the main part. It has multiple stations which performs operation on the parts (like valve body assembly, vision inspection, testing, packaging). The main feature of the cell is exhibited using the programming, control and diagnosis of the robotic manipulator amongst other peripheral and safety devices. We call it as Multi Process robotic cell because it can be used for more than one manufacturing process.  There are different stations in the robotic cell: 1. Conveyor Station: Use to transfer the inventory tray or pallet to the place where robot can pick them.2. Assembly Station: All the parts (axisymmetric and prismatic parts) will be assembled and screwed on the valve body in this station. Along with that inventory scanning and vision inspection of the valve bodies will be done.3.Testing Station: All the pneumatic ports of the valve body will be tested in this station. If there is any leakage, valve body will be rejected.4. Hex-nut Assembly Station:- Hex nut will be assembled on the body in this station.5. Packaging Station: Assembled valve bodies will be placed in a box and packaging is done. A printing head will print the required details( type of bodies, number of bodies, date of assembly etc) on the box and then the box will be dispatched.  The video here demonstrates the standalone function of loading inventory using trays from conveyor station to assembly station. The tray has multiple arrays of different sized part but same variety. There are trays with 3/2, 5/2, 5/2 double solenoid, 5/3 solenoid valve body. The end-effector of the robot have more than one tool attachments. This is possible due to tool changer allowing robot to perform multiple actions like pick and place, part screwing, gripping via vacuum suction cup, bar code part scanning. Training Site Integration Simulation & Testing Prototyping <-Cyber Physical assembly line Robotic Welding Cell-> Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in To create user defined webpages to control and monitor different devices (Appliances,machines) present in CP lab. 01Remote Control and monitoring of assembly line, Compressor and MPRC 02Remote control and monitoring of domestic appliances (lights,fans,AC) 03User defined access to control the machines remotely 04Activity logging of users 05Remote Control of the training Kits to train people remotely  Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in Robotic Welding Cell demonstrates a flexible manufacturing cell meaning it has an inventory of its own, material handling system, variety of parts with dynamic-order.This welding cell can be a part of production line and act as standalone system.Use of single or multiple robots may involve use of a PLC also.This cell uses a saftey PLC.This cell has all these features and the basis of selection can be done in a way that the cell will be programmed, utilization of peripheral devices (proximity sensors, pneumatic cylinders), safety devices and Automatic Identification & Data Capturing (AIDC) devices like RFID, vision camera etc. The cell has following level of flexibility but the decision can be made on the basis of the flexibility required as per the following parameters:1. Variety of parts for simultaneous production2. Change of schedule3. Ready for new partWorkstations and layout design :Workstations in the cell are as per the following given applications and subject to following parameters : 1. Load/Unload stations for the cell: It is the area where the raw materials are incepted in/out of the cell.2. Inventory storage area: An inventory storage where the storage of the raw material is kept.3. Inspection area: An area where the material post operations are inspected for quality. This is kept as a separate unit away from the cell but it can be an integral part of the cell itself.4. Operations area: An area where the operations like welding is performed. In the cell , this area is more of a workspace since two robots are responsible for performing welding on the material. And it can be anywhere in the robot's workspace. 5. Layout design: It is an integral part before identifying the robots and environment preparation and also plays a key role in safety of the cell.CAD Design & Simulation:The 3D visualization and detailing of the cell is done using a designing software. The robotic path simulation software asists in defining the best possible, collision free environment that is going to be the basis of the layout design and all other hardware that are going to be installed in the cell. Apart from designing & fabrication of the cell it helps in performing the robot simulations to generate offline program, collision free environment (this is how the level of flexibility achieved) for robots at operating stage and generating Augmented Reality applications. AutomationIIoTRoboticsAugmented Reality 01Machine as a service 04Real-Time Health Monitoring of Assets in Factory 02Customer Order Proessing 03Job work and research Contact us today for implementing Call us for any query011-26582053, 8076197190 or email us oninfo@iafsm.in FSM is providing an exciting opportunity for 100 individuals to participate in a summer internship program. The program is entirely online and will run full-time from June 1, 2023, to July 31, 2023, for a duration of two months. The focus of the internship is on the theme of \Smart Factory,\ which is a rapidly growing industry. This internship program is an excellent chance for individuals to gain valuable knowledge and experience in this field. It is an excellent opportunity for those who are looking to expand their skills and gain practical experience in this exciting area. If you are interested, don't miss this chance to be a part of the program and enhance your career prospects. FSM is focussed on enabling people from becoming Smart to Samarth (capable) fraternity by creating awareness, step-wise learning and becoming an expert from domains driving Industry 4.0 info@iafsm.in Level 2 : After successful completion of Level-1"

In [71]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("twmkn9/distilbert-base-uncased-squad2")


In [72]:
tokens=tokenizer.encode_plus(context,add_special_tokens=True,max_length=512,truncation=True,padding="max_length")
tokens

{'input_ids': [101, 2012, 1042, 6491, 2057, 3073, 2536, 7396, 2578, 1998, 2731, 2104, 1996, 5884, 1997, 6047, 5814, 1012, 1996, 2578, 2097, 9585, 7846, 2000, 3231, 1998, 2139, 8569, 2290, 3471, 2077, 23658, 1996, 5576, 2012, 1996, 2609, 1004, 2036, 3499, 2068, 2000, 7551, 1998, 7601, 7103, 2618, 2007, 2019, 6413, 4666, 1997, 3115, 1998, 28749, 7300, 1012, 3192, 2005, 6047, 5814, 1006, 1042, 6491, 1007, 7126, 1010, 6753, 1010, 1998, 11791, 6047, 5814, 8474, 2005, 2796, 3068, 2000, 7409, 1010, 2801, 2618, 1010, 1998, 3046, 2041, 1999, 2037, 4264, 1012, 1042, 6491, 2036, 8704, 2005, 1037, 7570, 27348, 4547, 8882, 1998, 8066, 1011, 2311, 2565, 2083, 1037, 17026, 4297, 19761, 3508, 1998, 3831, 4044, 1012, 1042, 6491, 4813, 2003, 1037, 7570, 27348, 4132, 2005, 4346, 10047, 16862, 3512, 2731, 3325, 2083, 2444, 8921, 1010, 3784, 4083, 1010, 2444, 13616, 1010, 2444, 13625, 1004, 2969, 13823, 11110, 2006, 19512, 7801, 5025, 8051, 1012, 1042, 6491, 2003, 3579, 6924, 2006, 12067, 2111, 2013, 3352,

In [169]:
tokens=tokenizer.encode_plus(context,add_special_tokens=False,return_tensors="pt")

In [74]:
tokens

{'input_ids': tensor([[2012, 1042, 6491,  ..., 2504, 1011, 1015]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [170]:
len(tokens['input_ids'][0])

31579

In [171]:
input_ids_chunks=tokens['input_ids'][0].split(510)
mask_chunks=tokens['attention_mask'][0].split(510)

for tensor in input_ids_chunks:
    print(len(tensor))
   

510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
469


In [172]:
import torch

chunksize=512
input_ids_chunks= list(input_ids_chunks)
mask_chunks= list(mask_chunks)
print(len(input_ids_chunks))
for i in range(len(input_ids_chunks)):
    input_ids_chunks[i]=torch.cat([torch.Tensor([101]),input_ids_chunks[i],torch.Tensor([102])])
    mask_chunks[i]=torch.cat([torch.Tensor([1]),mask_chunks[i],torch.Tensor([1])])
    
    pad_len=chunksize-input_ids_chunks[i].shape[0]
    if pad_len>0:
        input_ids_chunks[i]=torch.cat([input_ids_chunks[i],torch.Tensor([0]* pad_len)])
        mask_chunks[i]=torch.cat([mask_chunks[i],torch.Tensor([0]* pad_len)])

for chunk in input_ids_chunks:
    print(len(chunk))

62
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512


In [173]:
input_ids=torch.stack(input_ids_chunks)
attention_mask=torch.stack(mask_chunks)

input_dict={
    "input_ids":input_ids.long(),
    "attention_mask":attention_mask.int()
}
print(len(input_dict["input_ids"]))
input_dict

62


{'input_ids': tensor([[ 101, 2012, 1042,  ..., 2219, 4264,  102],
         [ 101, 1012, 1996,  ..., 4955, 1012,  102],
         [ 101, 6185, 4305,  ..., 1998, 6994,  102],
         ...,
         [ 101, 2864, 2478,  ..., 7082, 3630,  102],
         [ 101, 3593, 1010,  ..., 1996, 3526,  102],
         [ 101, 2021, 2009,  ...,    0,    0,    0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}

In [180]:
import torch
def context_conversion(question,context):
    
    tokens=tokenizer.encode_plus(question,context,add_special_tokens=False,return_tensors="pt")
    input_ids_chunks=tokens['input_ids'][0].split(509)
    mask_chunks=tokens['attention_mask'][0].split(509)
    
    input_ids_chunks= list(input_ids_chunks)
    mask_chunks= list(mask_chunks)
    
    tensor1=input_ids_chunks[0]
    tensor2=torch.Tensor([102])
    given_word=torch.Tensor([1029])
    index = (tensor1 == given_word).nonzero()
    index = index[0]
    tensor1_part1 = tensor1[:index+1]
    tensor1_part2 = tensor1[index+1:]
    input_ids_chunks[0] = torch.cat((tensor1_part1, tensor2, tensor1_part2), dim=0)

    
    tensor_mask1=mask_chunks[0]
    tensor_mask2=torch.Tensor([1])
    index = index[0]
    tensor1_mask_part1 = tensor_mask1[:index+1]
    tensor1_mask_part2 = tensor_mask1[index+1:]
    mask_chunks[0] = torch.cat((tensor1_mask_part1, tensor_mask2, tensor1_mask_part2), dim=0)

   

    chunksize=512
    # input_ids_chunks= list(input_ids_chunks)
    # mask_chunks= list(mask_chunks)
    
    
    for i in range(len(input_ids_chunks)):
        input_ids_chunks[i]=torch.cat([torch.Tensor([101]),input_ids_chunks[i],torch.Tensor([102])])
        mask_chunks[i]=torch.cat([torch.Tensor([1]),mask_chunks[i],torch.Tensor([1])])
        
        pad_len=chunksize-input_ids_chunks[i].shape[0]
        if pad_len>0:
            input_ids_chunks[i]=torch.cat([input_ids_chunks[i],torch.Tensor([0]* pad_len)])
            mask_chunks[i]=torch.cat([mask_chunks[i],torch.Tensor([0]* pad_len)])
    

    input_ids=torch.stack(input_ids_chunks)
    attention_mask=torch.stack(mask_chunks)
    
    input_dict={
        "input_ids":input_ids.long(),
        "attention_mask":attention_mask.int()
    }
    return input_dict

In [195]:
question="what is Hex-nut Assembly Station?"

inputs = context_conversion(question, context)
print(inputs)
input_ids = inputs['input_ids']
print(len(input_ids))
attention_mask = inputs['attention_mask']
tokenizer.decode(input_ids[0])


with torch.no_grad():
    output = model(input_ids, attention_mask=attention_mask)
print("hey")
answer_start = torch.argmax(output.start_logits)
print(answer_start)
answer_start_i=int(answer_start/512)
answer_start_j=int(answer_start%512)
answer_end = torch.argmax(output.end_logits)
answer_end_j=int(answer_end%512)
print(answer_end)


# # answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end + 1]))   
# # print(answer) 


{'input_ids': tensor([[  101,  2054,  2003,  ...,  8474,  2005,   102],
        [  101,  2796,  3068,  ..., 18384,   102,     0],
        [  101,  1042,  6491,  ...,  2473,   102,     0],
        ...,
        [  101,  3058,  1997,  ...,  3703,   102,     0],
        [  101,  2005,  1996,  ...,  1011,   102,     0],
        [  101,  7968,  4083,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}
63
hey
tensor(19520)
tensor(19627)


In [196]:
answer_start_i=int(answer_start/512)
answer_start_j=int(answer_start%512)

answer_end_j=int(answer_end%512)
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start_i][answer_start_j:answer_end_j + 1]))   
print(answer) 

visitors were given live demonstrations of technologies created for smart manufacturing in the cyber - physical lab at the indian institute of technology, delhi. here are some of the day ‘ s highlights. event date10th december, 2022 we are happy to report that on december 10, 2022, iitd - aia foundation for smart manufacturing presented various industry 4. 0 technologies at industry day at indian institute of technology, delhi. live demonstrations of technologies developed in smart manufacturing were given to the visitors. here are some of the day ‘ s highlights


In [197]:
question="what is CEFC?"

inputs = context_conversion(question, context)
print(inputs)
input_ids = inputs['input_ids']
print(len(input_ids))
attention_mask = inputs['attention_mask']
tokenizer.decode(input_ids[0])


with torch.no_grad():
    output = model(input_ids, attention_mask=attention_mask)
print("hey")
answer_start = torch.argmax(output.start_logits)
print(answer_start)
answer_start_i=int(answer_start/512)
answer_start_j=int(answer_start%512)
answer_end = torch.argmax(output.end_logits)
answer_end_j=int(answer_end%512)
print(answer_end)

{'input_ids': tensor([[  101,  2054,  2003,  ...,  1004,  3046,   102],
        [  101,  2068,  2041,  ...,  2099,   102,     0],
        [  101, 12098,  1042,  ...,  8066,   102,     0],
        ...,
        [  101,  1007,  2006,  ...,  1024,   102,     0],
        [  101,  2009,  2003,  ...,  3352,   102,     0],
        [  101,  2019,  6739,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}
63
hey
tensor(19463)
tensor(309)


In [198]:
answer_start_i=int(answer_start/512)
answer_start_j=int(answer_start%512)

answer_end_j=int(answer_end%512)
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start_i][answer_start_j:answer_end_j + 1]))   
print(answer) 

event date22th december, 2022 we are happy to inform that on december 22, 2022, iitd - aia foundation for smart manufacturing presented various industry 4. 0 technologies at workshop on automation for smart manufacturing organized by automation industry association in pune. visitors were given live demonstrations of technologies created for smart manufacturing in the cyber - physical lab at the indian institute of technology, delhi. here are some of the day ‘ s highlights. event date10th december, 2022 we are happy to report that on december 10, 2022, iitd - aia foundation for smart manufacturing presented various industry 4. 0 technologies at industry day at indian institute of technology, delhi. live demonstrations of technologies developed in smart manufacturing were given to the visitors. here are some of the day ‘ s highlights. event date29th november, 2022 mode of learning offline ( 4pm to 6pm ) iitd - aia foundation for smart manufacturing, takes immense pleasure to invite engin

In [199]:
def get_answers(question,context):
    inputs = context_conversion(question, context)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    
    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)

    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    answer_start_i=int(answer_start/512)
    answer_start_j=int(answer_start%512)

    answer_end_j=int(answer_end%512)
        
    
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start_i][answer_start_j:answer_end_j + 1]))    
    return answer

In [200]:
question="what is CEFC?"
print(f"Predicted Answer: {get_answers(question,context)}")

Predicted Answer: event date22th december, 2022 we are happy to inform that on december 22, 2022, iitd - aia foundation for smart manufacturing presented various industry 4. 0 technologies at workshop on automation for smart manufacturing organized by automation industry association in pune. visitors were given live demonstrations of technologies created for smart manufacturing in the cyber - physical lab at the indian institute of technology, delhi. here are some of the day ‘ s highlights. event date10th december, 2022 we are happy to report that on december 10, 2022, iitd - aia foundation for smart manufacturing presented various industry 4. 0 technologies at industry day at indian institute of technology, delhi. live demonstrations of technologies developed in smart manufacturing were given to the visitors. here are some of the day ‘ s highlights. event date29th november, 2022 mode of learning offline ( 4pm to 6pm ) iitd - aia foundation for smart manufacturing, takes immense pleasu

In [201]:
question="who are the contributors ?"
print(f"Predicted Answer: {get_answers(question, context)}")

Predicted Answer: expert faculty members from various departments contribute to it. fsm has highly trained manpower to implement the industry 4. 0 projects in guidance with experienced faculty members and industry partners


In [202]:
question="what is Hex-nut Assembly Station ?"
print(f"Predicted Answer: {get_answers(question, context)}")


Predicted Answer: visitors were given live demonstrations of technologies created for smart manufacturing in the cyber - physical lab at the indian institute of technology, delhi. here are some of the day ‘ s highlights. event date10th december, 2022 we are happy to report that on december 10, 2022, iitd - aia foundation for smart manufacturing presented various industry 4. 0 technologies at industry day at indian institute of technology, delhi. live demonstrations of technologies developed in smart manufacturing were given to the visitors. here are some of the day ‘ s highlights


In [203]:
question="what is the process for Internship 2023 ?"
print(f"Predicted Answer: {get_answers(question, context)}")


Predicted Answer: visitors were given live demonstrations of technologies created for smart manufacturing in the cyber - physical lab at the indian institute of technology, delhi. here are some of the day ‘ s highlights. event date10th december, 2022 we are happy to report that on december 10, 2022, iitd - aia foundation for smart manufacturing presented various industry 4. 0 technologies at industry day at indian institute of technology, delhi. live demonstrations of technologies developed in smart manufacturing were given to the visitors. here are some of the day ‘ s highlights
